# Initialization

## Library installation (only first time and in a local setup)

In [19]:
!pip cache purge

Files removed: 0


In [20]:
!pip install pygments >=2.4.1

In [21]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [22]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable


In [23]:
!pip install librosa

Defaulting to user installation because normal site-packages is not writeable


In [24]:
!pip install soundfile

Defaulting to user installation because normal site-packages is not writeable


In [25]:
!pip install --upgrade ipywidgets

Defaulting to user installation because normal site-packages is not writeable


In [26]:
!pip install packaging ninja

Defaulting to user installation because normal site-packages is not writeable


In [27]:
!pip install flash-attn --no-build-isolation

Defaulting to user installation because normal site-packages is not writeable


In [28]:
!pip install imblearn

Defaulting to user installation because normal site-packages is not writeable


In [29]:
!pip install mir_eval

Defaulting to user installation because normal site-packages is not writeable


In [30]:
!pip install --upgrade pygments 

Defaulting to user installation because normal site-packages is not writeable


In [31]:
!pip install metrics

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/841.7 kB ? eta -:--:--
   ---------------------------------------- 841.7/841.7 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: Pygments
    Found existing installation: Pygments 2.18.0
    Uninstalling Pygments-2.18.0:
      Successfully uninstalled Pygments-2.18.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 8.25.0 requires pygments>=2.4.0, but you have pygments 2.2.0 which is incompatible.
nbconvert 7.16.4 requires pygments>=2.4.1, but you have pygments 2.2.0 which is incompatible.


In [32]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


## Import libraries

In [33]:
from transformers import AutoFeatureExtractor, ASTForAudioClassification, ASTModel,Wav2Vec2Model,Wav2Vec2Config,Wav2Vec2Tokenizer,Wav2Vec2Processor
from datasets import load_dataset
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import torch
from functools import reduce
from operator import mul
import math
import torch.nn as nn
import librosa
from torch.utils.data import Dataset
import random
from tqdm.notebook import tqdm
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from matplotlib import pyplot as plt
import json
from scipy.signal import butter, lfilter
from scipy import stats
from collections import Counter
import torchaudio
import glob
from sklearn.metrics import f1_score,precision_score, recall_score,confusion_matrix
from torch.utils.data import DataLoader, TensorDataset
from torch.cuda.amp import autocast
from collections import defaultdict
from imblearn.over_sampling import SMOTE
import mir_eval
import scipy
import pandas as pd
import argparse
import os
import json
import numpy as np
import csv
import metrics
from datetime import datetime
import copy
from scipy import stats
from tqdm import tqdm

# Paths variables. Must be declared based on the directory location you use.

## Google Drive directory

In [34]:
drive=0

In [35]:
# Define root directory from Google Drive
if drive==1:
    from google.colab import drive
    drive.mount("/content/drive",force_remount=True)
    root_dir = "/content/drive/My Drive/Tesi/"

Json directories. The JSON must be already located here.

In [36]:
if(drive==0):
  train_json='C:/Users/aldob/Documents/GitHub/Thesis/Codice_Tesi/Json_files/train_data.json'
  val_json='C:/Users/aldob/Documents/GitHub/Thesis/Codice_Tesi/Json_files/val_data.json'
  train_dir='E:/Tesi/Development_Set/Training_Set/Audios'
  val_dir='E:/Tesi/Development_Set/Validation_Set/Audios'
else:
  train_json=os.path.join(root_dir, "Json_files/train_data.json")
  val_json=os.path.join(root_dir, "Json_files/val_data.json")
  train_dir=os.path.join(root_dir, "Training_Set/Audios")
  val_dir=os.path.join(root_dir, "Validation_Set/Audios")

Directories where you want to create/ are already created the episodes. 1 for spectrograms episodes and 1 for raw audio spectrograms.

In [37]:
if(drive==0):
  
  directory_path_spectrograms_episodes='E:/Tesi/episodes_pth'
  directory_path_training_stats='E:/Tesi/training_stats_pth'
 
  directory_path_inference_stats='E:/Tesi/inference_stats_pth'


  directory_path_spectrograms_inference='E:/Tesi/spectrograms_support_and_query_for_inference_pth'
  directory_path_spectrograms_inference_pcen='E:/Tesi/spectrograms_support_and_query_for_inference_pcen_pth'
  directory_path_spectrograms_inference_AST='E:/Tesi/spectrograms_support_and_query_for_inference_AST_16KHz_pcen_pth'
  directory_path_raw_audio_inference='E:/Tesi/raw_audios_support_and_query_for_inference_pth'

  

  

  training_directory_path_spectrograms_episodes="E:/Tesi/episodes_event_spectrogram_22KHz_pth"

  training_directory_path_raw_audio_episodes='E:/Tesi/episodes_raw_audio_pth'
  training_directory_path_spectrograms_episodes_AST="E:/Tesi/episodes_pth"
  training_directory_path_spectrograms_frame_negative_support_undersampling_episodes= "E:/Tesi/episodes_balanced_support_set_pth"
  training_directory_path_spectrograms__all_negative_supports_episodes= "E:/Tesi/episodes_all_negative_support_pth"

else:
  training_directory_path_raw_audio_episodes=os.path.join(root_dir, "episodes_raw_audio_pth")
  
  training_directory_path_spectrograms_episodes=os.path.join(root_dir, "episodes_pth")
  training_directory_path_spectrograms_frame_negative_support_undersampling_episodes=os.path.join(root_dir, "episodes_balanced_support_set_pth")
  training_directory_path_spectrograms__all_negative_supports_episodes=os.path.join(root_dir, "episodes_all_negative_support_pth")


  directory_path_spectrograms_audio_inference=os.path.join(root_dir, "audios_inference_pth")

  directory_path_training_stats=(root_dir, "training_stats_pth")
  directory_path_inference_stats=(root_dir, "inference_stats_pth")



Directory where to save the models to avoid make more than once the training phase.

In [38]:
if(drive==0):
  checkpoint_path=("E:/Tesi/Models_pth")
else:
  checkpoint_path=os.path.join(root_dir, "Models_pth")

# Utilities

## Support Set and Query set dictionaries creation utilities

In [57]:
def count_number_class(set):
    i=0
    for classe,file in set.items():
        i=i+1
    return i

In [58]:
def count_number_events(set):
    event_classes={}
    for classe,file in set.items():
        i=0
        for audio,events in file.items():
            for evento in events:
                i=i+1
        event_classes[classe]=i
    return event_classes

In [59]:
def return_max_num_events(set):
    events=count_number_events(set)
    i=0
    for classe,event_number in events.items():
        if events[classe]>i:
            i=events[classe]
    return i

In [60]:
def return_number_frames(set,durata_frame_in_secondi,frame_length=1,hop_length=1):
    num_frames_min=0
    num_frames_max=0
    for classe,file in set.items():
        for audio,events in file.items():
             for evento in events:
                intervallo=evento['end']-evento['start']
                sovrapposizione=hop_length/frame_length #quanto effettivamente le frame si sovrappongono l'un l'altra
                num_frames_evento=intervallo/(durata_frame_in_secondi*sovrapposizione)              #durata di 0.01 se frame=10ms.
                if(num_frames_evento>num_frames_max):
                    num_frames_max=num_frames_evento
                if (num_frames_min==0):
                    num_frames_min=num_frames_evento
                elif (num_frames_min>num_frames_evento):
                    num_frames_min=num_frames_evento
    return num_frames_max,num_frames_min

## Labels utilities

In [61]:
def return_audios_list(set):
    audio_list=[]
    for key in set:
            for file_name in set[key]:
                if(file_name not in audio_list):
                    audio_list.append(file_name)
    return audio_list

In [62]:
def get_classname_from_label_index(dictionary, label):
    for key, val in dictionary.items():
        if val == label:
            return key
    return None

## Support and query set creation utilities

In [63]:
def get_frames(audio, frame_length, hop_length):
    frames = []
    num_samples = len(audio)
    for starter in range(0, num_samples-frame_length + 1, hop_length):
        end = starter + frame_length
        frames.append(audio[starter:end])
    return frames

In [64]:
def conta_episodi(set):
    num_episodi=0
    for classe,struct in set.items():
        num_episodi+=len(struct)
    return num_episodi



## Conversion utility to get the file names in a directory. This is done to verify if all the episodes training are available.

In [65]:
def get_episodes_files_name(directory_path):

    file_pattern = os.path.join(directory_path, '*.pth')
    pth_files = glob.glob(file_pattern)


    pth_file_names = [os.path.basename(file) for file in pth_files]

    return pth_file_names

## Creation query and support set for AST and Wav2Vec

In [66]:
def load_audio(audio_path):
    audio, sample_rate = librosa.load(audio_path, sr=16000)
    return audio, sample_rate

In [67]:
# this function returns the audio batch of raw audio in bath audio preprocessed
def feature_extractor_events(events,feature_extractor):
    #feature_extractor = AutoFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
    batch_feature_extractor = []
    # for each audio in the batch
    for index in range(0, len(events)):
        # compute spectogram
        output = feature_extractor(events[index], sampling_rate=16000, return_tensors="pt")
        # store spectogram
        batch_feature_extractor.append((output))
    # model wants in input a tensor with shape [num_batch, num_frame, num_mel]
    events= torch.stack(batch_feature_extractor) # stack all the feature extracted events 
    
    return events

In [68]:
# this function returns the audio batch of raw audio in bath audio preprocessed
def feature_extractor_frame(frames,feature_extractor):
    #feature_extractor = AutoFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
    frames_feature = []
    # for each audio in the batch
    for frame in frames:
        # compute spectogram
        output = feature_extractor(frame, sampling_rate=16000, return_tensors="pt")
        # store spectogram
        frames_feature.append((output))
    frames_extracted = torch.stack(frames_feature) # stack all the feature extracted events 
    
    return frames_extracted

## Event equalizer functions

In [69]:
def padding_equalizer(events_list):
   

    min=0
    max=0

    for event in events_list:
        length=len(event)
        if(min==0):min=length
        if(min>length): min=length
        if(max<length):max=length
    
    equalized_events=[]
    for event in events_list:
        event=torch.tensor(event)
        if event.size(0) < max:
            
            padding = torch.zeros(max - event.size(0),event.size(1), dtype=event.dtype)
            
            event = torch.cat([event, padding])
            
        else:
            pass
        equalized_events.append(event)
    return equalized_events


# Folder creation with all audios for training/inference

In [52]:
import os
import shutil

def unificatore_file_audio(root_directory):
    audios_directory = os.path.join(root_directory, "Audios")
    os.makedirs(audios_directory, exist_ok=True)
    for folder_name in os.listdir(root_directory):
        folder_path = os.path.join(root_directory, folder_name)
        
        if os.path.isdir(folder_path):
            
            for file_audio in os.listdir(folder_path):
                
                if file_audio.endswith('.wav'):
                    file_path = os.path.join(folder_path, file_audio)
                    destination_path=os.path.join(audios_directory, file_audio)
                    if not os.path.exists(destination_path):
                        shutil.copy(file_path, audios_directory)
                        print("File {} copiato con successo!".format(file_audio))


root = "E:/Tesi/Development_Set/Training_Set"  # Sostituire con il proprio percorso della directory principale
#unificatore_file_audio(root)
root="E:/Tesi/Development_Set/Validation_Set"
#unificatore_file_audio(root)

# Support and query dictionaries set creation

In [53]:
def get_support_set(path):
    class_list=[]
    csv_files={}
    audio_files={}
    class_dictionary={}
    with open(path, 'r') as file:
        dati = json.load(file)

    for classe, value in dati.items():
        class_list+={classe}
        i=0
    for classe in class_list:
        class_dictionary[classe]=i
        i=i+1


    for classe in class_list:
        for csv in dati[classe]:
            if classe in csv_files:
                csv_files[classe].append(csv)
            else:
                csv_files[classe] = [csv]
            if(dati[classe][csv]["Support"]!={}):
                              #verifico che ci siano eventi positivi in quell'audio della classe
                if classe not in audio_files:
                    audio_files[classe] = {}


                if dati[classe][csv]["Audio"] not in audio_files[classe]:
                    audio_files[classe][dati[classe][csv]["Audio"]] = {}
                #audio_files[classe][dati[classe][csv]["Audio"]]=dati[classe][csv]["Audio"]["Supports"] Se volessi un formato diverso con anche i nomi degli eventi(seg0,1 ect...)
                lista_start_e_end_time_eventi=[]
                for evento in dati[classe][csv]["Support"]:
                    lista_start_e_end_time_eventi.append(dati[classe][csv]["Support"][evento])

                audio_files[classe][dati[classe][csv]["Audio"]]=lista_start_e_end_time_eventi


    return audio_files




def get_query_set(path):
    class_list=[]
    csv_files={}
    audio_files={}
    
    with open(path, 'r') as file:
        dati = json.load(file)
    for classe, value in dati.items():
            class_list+={classe}
        #print(len(class_list))
    for classe in class_list:
        for csv in dati[classe]:
            if classe in csv_files:
                csv_files[classe].append(csv)
            else:
                csv_files[classe] = [csv]
            if(dati[classe][csv]["Query"]!={}):
                            #verifico che ci siano eventi positivi in quell'audio della classe
                if classe not in audio_files:
                    audio_files[classe] = {}

                if dati[classe][csv]["Audio"] not in audio_files[classe]:
                    audio_files[classe][dati[classe][csv]["Audio"]] = {}
                #audio_files[classe][dati[classe][csv]["Audio"]]=dati[classe][csv]["Audio"]["Supports"] Se volessi un formato diverso con anche i nomi degli eventi(seg0,1 ect...)
                lista_start_e_end_time_eventi=[]
                for evento in dati[classe][csv]["Query"]:
                    lista_start_e_end_time_eventi.append(dati[classe][csv]["Query"][evento])

                audio_files[classe][dati[classe][csv]["Audio"]]=lista_start_e_end_time_eventi
    return audio_files

In [54]:
support_set=get_support_set(train_json)
print(support_set)
query_set=get_query_set(train_json)
number_classes_support_set=count_number_class(support_set)
number_events_support_set=count_number_events(support_set)
max_number_events=return_max_num_events(support_set)
num_max_frame,num_min_frame=return_number_frames(support_set,0.01)
print(number_classes_support_set,number_events_support_set,max_number_events)
print(num_max_frame,num_min_frame)


{'GRN': {'e1.wav': [{'start': 1512.64, 'end': 1514.47}, {'start': 1515.84, 'end': 1517.56}, {'start': 1518.64, 'end': 1520.15}, {'start': 1521.24, 'end': 1522.71}, {'start': 1523.8, 'end': 1525.29}], 'h1.wav': [{'start': 1625.64, 'end': 1626.99}, {'start': 1629.18, 'end': 1630.88}, {'start': 1640.23, 'end': 1642.78}, {'start': 1647.07, 'end': 1649.21}, {'start': 1652.98, 'end': 1655.44}], 'y1.wav': [{'start': 1213.32, 'end': 1214.22}, {'start': 1215.68, 'end': 1217.09}, {'start': 1218.24, 'end': 1219.5}, {'start': 1220.5, 'end': 1221.77}, {'start': 1222.79, 'end': 1224.58}]}, 'GIG': {'e1.wav': [{'start': 1953.05, 'end': 1955.71}], 'h1.wav': [{'start': 2059.19, 'end': 2059.68}, {'start': 2061.96, 'end': 2063.16}, {'start': 2076.6, 'end': 2077.7}, {'start': 2079.09, 'end': 2079.9}, {'start': 2087.86, 'end': 2088.6}], 'y1.wav': [{'start': 297.91, 'end': 298.31}, {'start': 371.73, 'end': 373.15}, {'start': 1241.2, 'end': 1241.54}, {'start': 1242.24, 'end': 1242.57}, {'start': 1290.4, 'end'

# Training Data Labelling

## Data Labelling Functions definition

In [55]:
def label_dataset(support_set,query_set=0,audio_list=0,frame_length=0.01):
    labels=[[] for _ in range(len(audio_list))]
    audio_id=0
    class_dictionary={}
    class_dictionary['UNKNOWN']=0
    for audio in audio_list:
        audio_path=os.path.join(train_dir,audio)
        audio_loaded, sr=librosa.load(audio_path)
        length_in_seconds=len(audio_loaded) / sr
        number_of_frames=math.ceil(length_in_seconds/frame_length)
        for i in range(number_of_frames):
            labels[audio_id].append(0)
        label_classe=1
        for classe,file in support_set.items():
            for audio_name,eventi in file.items():
                if audio ==audio_name:
                    for event in eventi:
                        event_start=event['start']
                        event_end=event['end']
                        frame_start=int(event_start/frame_length)
                        frame_end=int(event_end/frame_length)

                        for i in range (frame_start,frame_end+1):
                            labels[audio_id][i]=label_classe
            class_dictionary[classe]=label_classe
            label_classe=label_classe+1
        
        if query_set!=0:
            label_classe=1
            for classe,file in query_set.items():
                for audio_name,eventi in file.items():
                    if audio == audio_name:
                        event_list=file[audio]
                        for event in event_list:
                            event_start=event['start']
                            event_end=event['end']
                            frame_start=int(event_start/frame_length)
                            frame_end=int(event_end/frame_length)

                            for i in range (frame_start,frame_end):
                                labels[audio_id][i]=label_classe


                class_dictionary[classe]=label_classe
                label_classe=label_classe+1


        audio_id=audio_id+1
    return  labels,class_dictionary


## Labelling

In [56]:
audio_list_training=return_audios_list(support_set)
labels,label_mapping=label_dataset(support_set,query_set,audio_list_training)

train_file_paths=[]
for audio in audio_list_training:
    train_file_paths.append(os.path.join(train_dir,audio))

label_mapping

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\aldob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\aldob\AppData\Local\Temp\ipykernel_11328\4210611774.py", line 2, in <module>
    labels,label_mapping=label_dataset(support_set,query_set,audio_list_training)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\aldob\AppData\Local\Temp\ipykernel_11328\1543620227.py", line 8, in label_dataset
    audio_loaded, sr=librosa.load(audio_path)
                     ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\aldob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\core\audio.py", line 176, in load
    y, sr_native = __soundfile_load(path, offset, duration, dtype)
 

In [32]:
for label in labels:
        count=Counter(label)
        print(count)

Counter({0: 352310, 3: 5396, 1: 2027, 2: 267})
Counter({0: 334710, 1: 11752, 2: 7798, 3: 5740})
Counter({0: 349325, 1: 6077, 2: 2917, 3: 1681})
Counter({0: 356235, 4: 3765})
Counter({0: 355732, 5: 4268})
Counter({0: 56439, 6: 3595})
Counter({0: 248928, 8: 11116, 10: 1004, 7: 386})
Counter({0: 186482, 8: 3175, 9: 2203, 7: 141, 10: 91})
Counter({0: 714061, 18: 3004, 13: 590, 15: 469, 11: 440, 16: 409, 14: 352, 19: 304, 17: 201, 12: 170})
Counter({0: 712699, 18: 4439, 16: 1258, 14: 546, 13: 457, 15: 231, 21: 170, 11: 105, 20: 95})
Counter({0: 704282, 15: 10696, 18: 2240, 16: 991, 11: 823, 20: 425, 13: 275, 21: 268})
Counter({0: 717006, 19: 1865, 12: 425, 17: 349, 18: 260, 14: 95})
Counter({0: 714528, 18: 3032, 16: 1015, 13: 590, 15: 559, 14: 151, 20: 125})


# Data augmentation


In [70]:
def add_reverb(y, sr):
    # Convoluzione with reverb impuls
    seed = np.random.randn(int(sr * 0.3))
    reverb = np.convolve(y, seed, mode='full')[:len(y)]
    return reverb

def distort(y, distortion_factor=0.3): #factor 0.1 or 0.3
    y_distorted = y * (1 + distortion_factor * np.sin(2 * np.pi * y))
    return y_distorted

def add_noise(y, noise_factor=0.002): #between 0.001 and 0.003
    noise = np.random.randn(len(y))
    y_noisy = y + noise_factor * noise
    return y_noisy

def butter_bandpass(lowcut, highcut, sr, order=5):
    nyq = 0.5 * sr
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def apply_filter(y, sr, lowcut=5000, highcut=11025):
    b, a = butter_bandpass(lowcut, highcut, sr, order=6)
    y_filtered = lfilter(b, a, y)
    return y_filtered


def time_stretch(y, rate=1.2):
    y_stretched = librosa.effects.time_stretch(y, rate=rate)
    return y_stretched

def time_mask(spec, mask_param=1):  #between 0.5 and 1
    if isinstance(spec, np.ndarray):
        spec = torch.tensor(spec, dtype=torch.float32)

    return torchaudio.transforms.TimeMasking(time_mask_param=mask_param)(spec)

def frequency_mask(spec, mask_param=20): #between 10 and 20
    if isinstance(spec, np.ndarray):
        spec = torch.tensor(spec, dtype=torch.float32)
    return torchaudio.transforms.FrequencyMasking(freq_mask_param=mask_param)(spec)

def preemphasis_filter(y, coef=0.97):
    # Applica il filtro di preenfasi
    y_filtered = np.append(y[0], y[1:] - coef * y[:-1])
    return y_filtered


In [71]:
def no_data_aug(y):
    return y

def augment_audio(y, sr, n_versions=19):
    augmented_versions = []
    for _ in range(n_versions):
        y_aug = np.copy(y)

        # Lista delle possibili trasformazioni
        transformations = [
            lambda y: add_reverb(y, sr),
            lambda y: distort(y, distortion_factor=random.uniform(0.1, 0.3)),
            lambda y: add_noise(y, noise_factor=random.uniform(0.001, 0.003)),
            lambda y: apply_filter(y, sr, lowcut=random.uniform(300, 1000), highcut=random.uniform(2000, 8000)),
            lambda y: time_stretch(y, rate=random.uniform(0.8, 1.2)),
            lambda y:preemphasis_filter(y,coef=random.uniform(0.94,0.98))

        ]

        # Creation of augmented data which at least 4 of the 7 trasformations.
        selected_transformations = random.sample(transformations, k=random.randint(4, len(transformations)))


        for transform in selected_transformations:
            y_aug = transform(y_aug)

        augmented_versions.append(y_aug)

    return augmented_versions




def augment_spectrogram(spectrogram):
    spectrogram_aug=spectrogram

    random_float = random.random()
    if random_float < 0.33:
        spectrogram_aug=time_mask(spectrogram_aug)
        spectrogram_aug=frequency_mask(spectrogram_aug)
    elif 0.33 <= random_float < 0.66:
        spectrogram_aug=time_mask(spectrogram_aug)
    else:
        spectrogram_aug=frequency_mask(spectrogram_aug)

    return spectrogram_aug



# Training Support and query set creation for each episode (for each audio 3/4 episodes usually). Total 54 episodes for training set

### Support and query creation for from scratch model. The spectrograms are created and augmented based from a single frame. All negatives support frames taken in the support set.

In [32]:
def create_support_and_query_sets_all_negative_support(labels,file_paths, support_set_dictionary,episodi_non_presenti,num_support_per_class=5):
    episodes = [] #episodes of training for the prototypical network. Considering an audio with 4 classes in it, we want 4 episodes.
    j=1
    #from the label[audio][frame] structure it returns a sequence of tuples. For each audio file it corrisponds the labels for the audio.

    for file_id,label_list in enumerate(labels):

        #dictionary which, for each label values contains every occurance of the label in a (file,label_id) format.
        #We'll use it to create the query and support_set.


            #label_name=get_classname_from_label_index(label_mapping,label)
            audio_name=audio_list_training[file_id]
            path=train_file_paths[file_id]
            #audio_raw,sr = librosa.load(path)
            if not os.path.exists(training_directory_path_spectrograms__all_negative_supports_episodes):
                os.makedirs("episodes_all_negative_support_pth")

            print(f" Inizio creazione support e query per l'audio: {audio_name}")

            lista_classi_in_audio=set()
            for label in label_list:
                  if(label!=0):
                    lista_classi_in_audio.add(label)
            for i in lista_classi_in_audio:
                label_name=get_classname_from_label_index(label_mapping,i)
                if(j not in episodi_non_presenti):
                    print(f" Episodio {j} già presente, si passa al successivo")
                    j=j+1
                    continue
                audio_raw,sr = librosa.load(path)
                for classe,struct in support_set_dictionary.items():
                            for audio,eventi in struct.items():

                                support_spectrograms=[]
                                negative_support_spectrograms=[]
                                if (classe==label_name and audio==audio_name):
                                    events=support_set_dictionary[classe][audio]
                                    events=events[:num_support_per_class]
                                    end_support=0
                                    start_negative_support_set=0
                                    for evento in events:
                                        start=int(evento['start']*sr)
                                        end=int(evento['end']*sr)

                                        #creation support set spectrograms for "unknown" class.
                                        audio_event=audio_raw[start_negative_support_set:start]
                                        negative_frames=get_frames(audio_event,220,220)
                                        for frame in negative_frames:
                                            spectrogram_frame=librosa.feature.melspectrogram(y=frame,n_fft=220,hop_length=220,n_mels=128)
                                            negative_support_spectrograms.append((spectrogram_frame,0))



                                        start_negative_support_set=end+1
                                        if(end>end_support):
                                            end_support=end

                                        #audio_portion containing the event for the support set
                                        audio_event=audio_raw[start:end]

                                        frames=get_frames(audio_event,220,220) #without overlapping, hop_length=frame_lenght
                                        spectrograms=[]
                                        for frame in frames:
                                            spectrogram_frame=librosa.feature.melspectrogram(y=frame,n_fft=220,hop_length=220,n_mels=128)
                                            spectrograms.append((spectrogram_frame,i))
                                            spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                                            spectrograms.append((spectrogram_augmented,i))




                                        #data augmentation section for raw audios, the objective is increasing nearly 20x fold the support dimension
                                        audios_augmented=augment_audio(audio_event,sr)
                                        for audio_augmented in audios_augmented:
                                            frames=get_frames(audio_augmented,220,220)
                                            for frame in frames:
                                                spectrogram_frame=librosa.feature.melspectrogram(y=frame,n_fft=220,hop_length=220,n_mels=128)
                                                #le frame me le faccio creare direttamente dal mel
                                                spectrograms.append((spectrogram_frame,i))
                                                spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                                                spectrograms.append((spectrogram_augmented,i))



                                        support_spectrograms.extend(spectrograms)

                                    support_spectrograms.extend((negative_support_spectrograms))



                                    audio_query=audio_raw[end_support+220:]
                                    frames=get_frames(audio_query,220,220)
                                    spectrograms_query=[]

                                    k=0
                                    print(len(frames))
                                    print(len(label_list)-(end_support/22050)/0.01)

                                    for frame_id in range(int((end_support/22050)/0.01),len(label_list)):
                                        spectrogram_frame=librosa.feature.melspectrogram(y=frames[k],n_fft=220,hop_length=220,n_mels=128)
                                        spectrograms_query.append((spectrogram_frame,labels[file_id][frame_id]))
                                        k=k+1
                                    episode=(i,support_spectrograms,spectrograms_query)
                                    #saving the episode in a file.
                                    file_name=f'training_episode {j}.pth'
                                    file_path=os.path.join(training_directory_path_spectrograms__all_negative_supports_episodes, file_name)
                                    torch.save(episode,file_path)
                                    episodes.append(episode)

                                    print(f"Finito episodio,salvataggio con successo: {j}")
                                    j=j+1

    return 1

### Support and query creation for from scratch model. The spectrograms are created and augmented based from a single frame. Negatives support frames  of the same dimension of positive support frame taken in the support set.

In [33]:
def create_support_and_query_sets_balanced_negative_support(labels,file_paths, support_set_dictionary,episodi_non_presenti,num_support_per_class=5):
    episodes = [] #episodes of training for the prototypical network. Considering an audio with 4 classes in it, we want 4 episodes.
    j=1
    #from the label[audio][frame] structure it returns a sequence of tuples. For each audio file it corrisponds the labels for the audio.

    for file_id,label_list in enumerate(labels):

        #dictionary which, for each label values contains every occurance of the label in a (file,label_id) format.
        #We'll use it to create the query and support_set.


            #label_name=get_classname_from_label_index(label_mapping,label)
            audio_name=audio_list_training[file_id]
            path=train_file_paths[file_id]
            #audio_raw,sr = librosa.load(path)
            if not os.path.exists(training_directory_path_spectrograms_frame_negative_support_undersampling_episodes):
                os.makedirs("episodes_balanced_support_set_pth")

            print(f" Inizio creazione support e query per l'audio: {audio_name}")

            lista_classi_in_audio=set()
            for label in label_list:
                  if(label!=0):
                    lista_classi_in_audio.add(label)
            for i in lista_classi_in_audio:
                label_name=get_classname_from_label_index(label_mapping,i)
                if(j not in episodi_non_presenti):
                    print(f" Episodio {j} già presente, si passa al successivo")
                    j=j+1
                    continue
                audio_raw,sr = librosa.load(path)
                for classe,struct in support_set_dictionary.items():
                            for audio,eventi in struct.items():

                                support_spectrograms=[]
                                negative_support_spectrograms=[]
                                if (classe==label_name and audio==audio_name):
                                    events=support_set_dictionary[classe][audio]
                                    events=events[:num_support_per_class]
                                    end_support=0
                                    start_negative_support_set=0
                                    for evento in events:
                                        start=int(evento['start']*sr)
                                        end=int(evento['end']*sr)

                                        #creation support set spectrograms for "unknown" class.
                                        audio_event=audio_raw[start_negative_support_set:start]
                                        negative_frames=get_frames(audio_event,220,220)
                                        for frame in negative_frames:
                                            spectrogram_frame=librosa.feature.melspectrogram(y=frame,n_fft=220,hop_length=220,n_mels=128)
                                            negative_support_spectrograms.append((spectrogram_frame,0))



                                        start_negative_support_set=end+1
                                        if(end>end_support):
                                            end_support=end

                                        #audio_portion containing the event for the support set
                                        audio_event=audio_raw[start:end]

                                        frames=get_frames(audio_event,220,220) #without overlapping, hop_length=frame_lenght
                                        spectrograms=[]
                                        for frame in frames:
                                            spectrogram_frame=librosa.feature.melspectrogram(y=frame,n_fft=220,hop_length=220,n_mels=128)
                                            spectrograms.append((spectrogram_frame,i))
                                            spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                                            spectrograms.append((spectrogram_augmented,i))




                                        #data augmentation section for raw audios, the objective is increasing nearly 20x fold the support dimension
                                        audios_augmented=augment_audio(audio_event,sr)
                                        for audio_augmented in audios_augmented:
                                            frames=get_frames(audio_augmented,220,220)
                                            for frame in frames:
                                                spectrogram_frame=librosa.feature.melspectrogram(y=frame,n_fft=220,hop_length=220,n_mels=128)
                                                #le frame me le faccio creare direttamente dal mel
                                                spectrograms.append((spectrogram_frame,i))
                                                spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                                                spectrograms.append((spectrogram_augmented,i))



                                        support_spectrograms.extend(spectrograms)

                                    
                                    negative_samples_to_keep=len(support_spectrograms)
                                    print(negative_samples_to_keep)
                                    print(len(negative_support_spectrograms))
                                    if(len(negative_support_spectrograms)>=negative_samples_to_keep):
                                        negative_support_spectrograms=random.sample(negative_support_spectrograms,negative_samples_to_keep)
                                    
                                    support_spectrograms.extend((negative_support_spectrograms))



                                    audio_query=audio_raw[end_support+220:]
                                    frames=get_frames(audio_query,220,220)
                                    spectrograms_query=[]

                                    k=0
                                    print(len(frames))
                                    print(len(label_list)-(end_support/22050)/0.01)

                                    for frame_id in range(int((end_support/22050)/0.01),len(label_list)):
                                        spectrogram_frame=librosa.feature.melspectrogram(y=frames[k],n_fft=220,hop_length=220,n_mels=128)
                                        spectrograms_query.append((spectrogram_frame,labels[file_id][frame_id]))
                                        k=k+1
                                    episode=(i,support_spectrograms,spectrograms_query)
                                    #saving the episode in a file.
                                    file_name=f'training_episode {j}.pth'
                                    file_path=os.path.join(training_directory_path_spectrograms_frame_negative_support_undersampling_episodes, file_name)
                                    torch.save(episode,file_path)
                                    episodes.append(episode)

                                    print(f"Finito episodio,salvataggio con successo: {j}")
                                    j=j+1

    return 1

### Support and query creation for from scratch model. The spectrograms are created and augmented based from an event /audio portion. All negatives frames of negative support set  taken in the support set.

In [34]:
def create_support_and_query_sets_with_new_spectrogram_creation(labels,file_paths, support_set_dictionary,episodi_non_presenti,num_support_per_class=5):
    episodes = [] #episodes of training for the prototypical network. Considering an audio with 4 classes in it, we want 4 episodes.
    j=1
    #from the label[audio][frame] structure it returns a sequence of tuples. For each audio file it corrisponds the labels for the audio.

    for file_id,label_list in enumerate(labels):

        #dictionary which, for each label values contains every occurance of the label in a (file,label_id) format.
        #We'll use it to create the query and support_set.


            
            audio_name=audio_list_training[file_id]
            path=train_file_paths[file_id]
            
            if not os.path.exists(training_directory_path_spectrograms_episodes):
                os.makedirs(training_directory_path_spectrograms_episodes)

            print(f" Inizio creazione support e query per l'audio: {audio_name}")

            lista_classi_in_audio=set()
            for label in label_list:
                  if(label!=0):
                    lista_classi_in_audio.add(label)
            for i in lista_classi_in_audio:
                label_name=get_classname_from_label_index(label_mapping,i)
                if(j not in episodi_non_presenti):
                    print(f" Episodio {j} già presente, si passa al successivo")
                    j=j+1
                    continue
                audio_raw,sr = librosa.load(path)
                for classe,struct in support_set_dictionary.items():
                            for audio,eventi in struct.items():

                                support_spectrograms=[]
                                negative_support_spectrograms=[]
                                if (classe==label_name and audio==audio_name):
                                    events=support_set_dictionary[classe][audio]
                                    events=events[:num_support_per_class]
                                    end_support=0
                                    start_negative_support_set=0

                                    mean_duration=0
                                    for evento in events:
                                        start=int(evento['start']*sr)
                                        end=int(evento['end']*sr)
                                        if mean_duration==0: mean_duration=end-start
                                        else: mean_duration+=end-start
                                    mean_duration=mean_duration/5

                                    for evento in events:
                                        start=int(evento['start']*sr)
                                        end=int(evento['end']*sr)

                                        #creation support set spectrograms for "unknown" class.
                                        audio_event=audio_raw[start_negative_support_set:start]
                                        negative_frames=get_frames(audio_event,int(mean_duration),int(mean_duration))
                                        
                                        for negative_frame in  negative_frames:
                                            spectrogram_frames=librosa.feature.melspectrogram(y=negative_frame,n_fft=220,hop_length=110,n_mels=128)
                                        
                                            for l in range(0,spectrogram_frames.shape[1],2):
                                                end_frame = l + 2
                                                if end_frame <= spectrogram_frames.shape[1]:
                                                    spectrogram_frame = spectrogram_frames[:, l:end_frame]
                                                    negative_support_spectrograms.append((spectrogram_frame,0))
                                                 


                                        start_negative_support_set=end+1
                                        if(end>end_support):
                                            end_support=end

                                        #audio_portion containing the event for the support set
                                        audio_event=audio_raw[start:end]
                                        spectrograms=[]
                                        frames=librosa.feature.melspectrogram(y=audio_event,n_fft=220,hop_length=110,n_mels=128) 
        
                                        for  l in range(0,frames.shape[1],2):
                                            end_frame = l + 2
                                            if end_frame <= frames.shape[1]:
                                                spectrogram_frame = frames[:, l:end_frame]
                                                
                                                spectrograms.append((spectrogram_frame,i))
                                                spectrogram_frame=torch.tensor(spectrogram_frame)
                                                spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                                                spectrograms.append((spectrogram_augmented,i))




                                        #data augmentation section for raw audios, the objective is increasing nearly 20x fold the support dimension
                                        audios_augmented=augment_audio(audio_event,sr)
                                        for audio_augmented in audios_augmented:
                                            spectrogram_frames=librosa.feature.melspectrogram(y=audio_augmented,n_fft=220,hop_length=110,n_mels=128)
                                            for  l in range(0,spectrogram_frames.shape[1],2):
                                                end_frame = l + 2
                                                if end_frame <= spectrogram_frames.shape[1]:
                                                    spectrogram_frame = spectrogram_frames[:, l:end_frame]      
                                                    spectrograms.append((spectrogram_frame,i))
                                                    spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                                                    spectrograms.append((spectrogram_augmented,i))


                                        support_spectrograms.extend(spectrograms)

                                    """
                                    negative_samples_to_keep=len(support_spectrograms)
                                    print(negative_samples_to_keep)
                                    print(len(negative_support_spectrograms))
                                    if(len(negative_support_spectrograms)>=negative_samples_to_keep):
                                        negative_support_spectrograms=random.sample(negative_support_spectrograms,negative_samples_to_keep)
                                    """
                                    support_spectrograms.extend((negative_support_spectrograms))



                                    audio_query=audio_raw[end_support+220:]
                                    
                                    frames=get_frames(audio_query,int(mean_duration),int(mean_duration))
                                    query_spectrograms=[]
                                    z=len(labels[file_id])
                                    print(z)
                                    k=int((end_support/22050)/0.01)
                                    x=0
                                    frames=get_frames(audio_query,int(mean_duration),int(mean_duration))

                                    for frame in frames:
                                        spectrogram_frames=librosa.feature.melspectrogram(y=frame,n_fft=220,hop_length=110,n_mels=128) #labels must go from labels(k) to labels(k+)
                                        for l in range(0,spectrogram_frames.shape[1],2):
                                            end_frame = l + 2
                                            if end_frame <= spectrogram_frames.shape[1]:
                                                spectrogram_frame = spectrogram_frames[:, l:end_frame]      
                                                x+=1
                                                
                                                if(x%2)==0:
                                                    if(k>z):break
                                                    query_spectrograms.append((spectrogram_frame,labels[file_id][k]))
                                                    
                                                    k+=1
                                                else: 
                                                    if(k>z):break
                                                    
                                                    if labels[file_id][k]!=labels[file_id][k+1]: 
                                                        query_spectrograms.append((spectrogram_frame,i)) #there is a frame 50% 0 and 50% 1, must be choosen if put 0 or 1.
                                                    else: 
                                                        query_spectrograms.append((spectrogram_frame,labels[file_id][k]))

                                    episode=(i,support_spectrograms,query_spectrograms)
                                    print(len(episode[1]))
                                    print(len(episode[2]))
                                    #saving the episode in a file.
                                    file_name=f'training_episode {j}.pth'
                                    file_path=os.path.join(training_directory_path_spectrograms_episodes, file_name)
                                    torch.save(episode,file_path)
                                    episodes.append(episode)

                                    print(f"Finito episodio,salvataggio con successo: {j}")
                                    j=j+1

    return 1

### Support and query Set Creation for the AST Model.The spectrograms are created and augmented based from an event /audio portion. All negatives frames of negative support set  taken in the support set. Sample rate of 16Khz as AudioSet. Sperimental use of the PCEN also in episodes creation and therefore for training

In [35]:
def create_support_and_query_sets_with_new_spectrogram_creationAST_16KHz(labels,file_paths, support_set_dictionary,episodi_non_presenti,num_support_per_class=5):
    episodes = [] #episodes of training for the prototypical network. Considering an audio with 4 classes in it, we want 4 episodes.
    j=1
    #from the label[audio][frame] structure it returns a sequence of tuples. For each audio file it corrisponds the labels for the audio.

    for file_id,label_list in enumerate(labels):

        #dictionary which, for each label values contains every occurance of the label in a (file,label_id) format.
        #We'll use it to create the query and support_set.

            
            audio_name=audio_list_training[file_id]
            path=train_file_paths[file_id]
            
            if not os.path.exists(training_directory_path_spectrograms_episodes_AST):
                os.makedirs(training_directory_path_spectrograms_episodes_AST)

            print(f" Inizio creazione support e query per l'audio: {audio_name}")

            lista_classi_in_audio=set()
            for label in label_list:
                  if(label!=0):
                    lista_classi_in_audio.add(label)
            for i in lista_classi_in_audio:
                label_name=get_classname_from_label_index(label_mapping,i)
                if(j not in episodi_non_presenti):
                    print(f" Episodio {j} già presente, si passa al successivo")
                    j=j+1
                    continue
                audio_raw,sr = librosa.load(path,sr=16000)
                for classe,struct in support_set_dictionary.items():
                            for audio,eventi in struct.items():

                                support_spectrograms=[]
                                negative_support_spectrograms=[]
                                if (classe==label_name and audio==audio_name):
                                    events=support_set_dictionary[classe][audio]
                                    events=events[:num_support_per_class]
                                    end_support=0
                                    start_negative_support_set=0

                                    mean_duration=0
                                    for evento in events:
                                        start=int(evento['start']*sr)
                                        end=int(evento['end']*sr)
                                        if mean_duration==0: mean_duration=end-start
                                        else: mean_duration+=end-start
                                    mean_duration=mean_duration/5

                                    for evento in events:
                                        start=int(evento['start']*sr)
                                        end=int(evento['end']*sr)

                                        #creation support set spectrograms for "unknown" class.
                                        audio_event=audio_raw[start_negative_support_set:start]
                                        negative_frames=get_frames(audio_event,int(mean_duration),int(mean_duration))
                                        
                                        for negative_frame in  negative_frames:
                                            spectrogram_frames=librosa.feature.melspectrogram(y=negative_frame,n_fft=220,hop_length=110,n_mels=128)
                                            spectrogram_frames = librosa.pcen(spectrogram_frames * (2**31))
                                        
                                            for l in range(0,spectrogram_frames.shape[1],2):
                                                end_frame = l + 2
                                                if end_frame <= spectrogram_frames.shape[1]:
                                                    spectrogram_frame = spectrogram_frames[:, l:end_frame]
                                                    negative_support_spectrograms.append((spectrogram_frame,0))
                                                 


                                        start_negative_support_set=end+1
                                        if(end>end_support):
                                            end_support=end

                                        #audio_portion containing the event for the support set
                                        audio_event=audio_raw[start:end]
                                        spectrograms=[]
                                        frames=librosa.feature.melspectrogram(y=audio_event,n_fft=220,hop_length=110,n_mels=128) 
                                        frames = librosa.pcen(spectrogram_frames * (2**31))
        
                                        for  l in range(0,frames.shape[1],2):
                                            end_frame = l + 2
                                            if end_frame <= frames.shape[1]:
                                                spectrogram_frame = frames[:, l:end_frame]
                                                
                                                spectrograms.append((spectrogram_frame,i))
                                                spectrogram_frame=torch.tensor(spectrogram_frame)
                                                spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                                                spectrograms.append((spectrogram_augmented,i))




                                        #data augmentation section for raw audios, the objective is increasing nearly 20x fold the support dimension
                                        audios_augmented=augment_audio(audio_event,sr)
                                        for audio_augmented in audios_augmented:
                                            spectrogram_frames=librosa.feature.melspectrogram(y=audio_augmented,n_fft=220,hop_length=110,n_mels=128)
                                            spectrogram_frames = librosa.pcen(spectrogram_frames * (2**31))
                                            for  l in range(0,spectrogram_frames.shape[1],2):
                                                end_frame = l + 2
                                                if end_frame <= spectrogram_frames.shape[1]:
                                                    spectrogram_frame = spectrogram_frames[:, l:end_frame]      
                                                    spectrograms.append((spectrogram_frame,i))
                                                    spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                                                    spectrograms.append((spectrogram_augmented,i))


                                        support_spectrograms.extend(spectrograms)

                                   
                                    support_spectrograms.extend((negative_support_spectrograms))



                                    audio_query=audio_raw[end_support+160:]
                                    
                                    frames=get_frames(audio_query,int(mean_duration),int(mean_duration))
                                    query_spectrograms=[]
                                    z=len(labels[file_id])
                                    print(z)
                                    k=int((end_support/16000)/0.01)
                                    x=0
                                    frames=get_frames(audio_query,int(mean_duration),int(mean_duration))

                                    for frame in frames:
                                        spectrogram_frames=librosa.feature.melspectrogram(y=frame,n_fft=220,hop_length=110,n_mels=128) #labels must go from labels(k) to labels(k+)
                                        spectrogram_frames = librosa.pcen(spectrogram_frames * (2**31))
                                        for l in range(0,spectrogram_frames.shape[1],2):
                                            end_frame = l + 2
                                            if end_frame <= spectrogram_frames.shape[1]:
                                                spectrogram_frame = spectrogram_frames[:, l:end_frame]      
                                                x+=1
                                                
                                                if(x%2)==0:
                                                    if(k>z):break
                                                    query_spectrograms.append((spectrogram_frame,labels[file_id][k]))
                                                    
                                                    k+=1
                                                else: 
                                                    if(k>z):break
                                                    
                                                    if labels[file_id][k]!=labels[file_id][k+1]: 
                                                        query_spectrograms.append((spectrogram_frame,i)) #there is a frame 50% 0 and 50% 1, must be choosen if put 0 or 1.
                                                    else: 
                                                        query_spectrograms.append((spectrogram_frame,labels[file_id][k]))

                                    episode=(i,support_spectrograms,query_spectrograms)
                                    print(len(episode[1]))
                                    print(len(episode[2]))
                                    #saving the episode in a file.
                                    file_name=f'training_episode {j}.pth'
                                    file_path=os.path.join(training_directory_path_spectrograms_episodes_AST, file_name)
                                    torch.save(episode,file_path)
                                    episodes.append(episode)

                                    print(f"Finito episodio,salvataggio con successo: {j}")
                                    j=j+1

    return 1

### Support and query set creation for raw audios. The raw audios can be used then for Wav2Vec model as inputs

In [36]:
def create_support_and_query_sets_raw_audio(labels,file_paths, support_set_dictionary,episodi_non_presenti,num_support_per_class=5):
    episodes = [] #episodes of training for the prototypical network. Considering an audio with 4 classes in it, we want 4 episodes.
    j=1
    #from the label[audio][frame] structure it returns a sequence of tuples. For each audio file it corrisponds the labels for the audio.

    for file_id,label_list in enumerate(labels):

        #dictionary which, for each label values contains every occurance of the label in a (file,label_id) format.
        #We'll use it to create the query and support_set.


            #label_name=get_classname_from_label_index(label_mapping,label)
            audio_name=audio_list_training[file_id]
            path=train_file_paths[file_id]
            #audio_raw,sr = librosa.load(path)
            if not os.path.exists(training_directory_path_raw_audio_episodes):
                os.makedirs("episodes_raw_audio_pth")

            print(f" Inizio creazione support e query per l'audio: {audio_name}")

            lista_classi_in_audio=set()
            for label in label_list:
                  if(label!=0):
                    lista_classi_in_audio.add(label)
            for i in lista_classi_in_audio:
                label_name=get_classname_from_label_index(label_mapping,i)
                if(j not in episodi_non_presenti):
                    print(f" Episodio {j} già presente, si passa al successivo")
                    j=j+1
                    continue
                audio_raw,sr = librosa.load(path,sr=16000)
                for classe,struct in support_set_dictionary.items():
                            for audio,eventi in struct.items():

                                support_raw_audios=[]
                                negative_support_raw_audios=[]
                                if (classe==label_name and audio==audio_name):
                                    events=support_set_dictionary[classe][audio]
                                    events=events[:num_support_per_class]
                                    end_support=0
                                    start_negative_support_set=0
                                    for evento in events:
                                        start=int(evento['start']*sr)
                                        end=int(evento['end']*sr)

                                        #creation support set raw audio for "unknown" class.
                                        audio_event=audio_raw[start_negative_support_set:start]
                                        negative_frames=get_frames(audio_event,160,160)
                                        for frame in negative_frames:
                                            negative_support_raw_audios.append((frame,0))



                                        start_negative_support_set=end+1
                                        if(end>end_support):
                                            end_support=end

                                        #audio_portion containing the event for the support set
                                        audio_event=audio_raw[start:end]

                                        frames=get_frames(audio_event,160,160) #without overlapping, hop_length=frame_lenght
                                        raw_audios=[]
                                        for frame in frames:

                                            raw_audios.append((frame,i))


                                        #data augmentation section for raw audios, the objective is increasing nearly 20x fold the support dimension
                                        audios_augmented=augment_audio(audio_event,sr)
                                        for audio_augmented in audios_augmented:
                                            frames=get_frames(audio_augmented,160,160)
                                            for frame in frames:
                                                raw_audios.append((frame,i))



                                        support_raw_audios.extend(raw_audios)

                              

                                    support_raw_audios.extend((negative_support_raw_audios))



                                    audio_query=audio_raw[end_support+160:]
                                    frames=get_frames(audio_query,160,160)
                                    raw_audio_query=[]
                                    print(len(frames))
                                    print(len(label_list)-(end_support/16000)/0.01)
                                    k=0

                                    for frame_id in range(int((end_support/16000)/0.01),len(frames)+int((end_support/16000)/0.01)):
                                        frame=frames[k]
                                        raw_audio_query.append((frame,labels[file_id][frame_id]))
                                        k=k+1
                                    episode=(i,support_raw_audios,raw_audio_query)
                                    #saving the episode in a file.
                                    file_name=f'training_episode {j}.pth'
                                    file_path=os.path.join(training_directory_path_raw_audio_episodes, file_name)
                                    torch.save(episode,file_path)
                                    episodes.append(episode)

                                    print(f"Finito episodio,salvataggio con successo: {j}")
                                    j=j+1

    return 1

In [ ]:
n= conta_episodi(support_set)
n

54

## Support and query Set creation for spectrograms + save

### Creation with all negative samples for support set

In [89]:
episodi_non_presenti=[]
for j in range(1,55):
    file_name=f'training_episode {j}.pth'
    file_path=os.path.join(training_directory_path_spectrograms__all_negative_supports_episodes, file_name)
    if os.path.exists(file_path):
       continue

    else:
        episodi_non_presenti.append(j)
print(episodi_non_presenti)


codice_ritorno= create_support_and_query_sets_all_negative_support(labels,train_file_paths,support_set,episodi_non_presenti=episodi_non_presenti, num_support_per_class=5)
if(codice_ritorno==1):
    print("Creazione conclusa")

[38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]
 Inizio creazione support e query per l'audio: e1.wav
 Episodio 1 già presente, si passa al successivo
 Episodio 2 già presente, si passa al successivo
 Episodio 3 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: h1.wav
 Episodio 4 già presente, si passa al successivo
 Episodio 5 già presente, si passa al successivo
 Episodio 6 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: y1.wav
 Episodio 7 già presente, si passa al successivo
 Episodio 8 già presente, si passa al successivo
 Episodio 9 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: n1.wav
 Episodio 10 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: a1.wav
 Episodio 11 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: sp4f1_2015_04_24-05_00_00_0002.wav
 Episodio 12 già presente, si passa al

### Creation with balanced support set

In [ ]:
episodi_non_presenti=[]
for j in range(1,55):
    file_name=f'training_episode {j}.pth'
    file_path=os.path.join(training_directory_path_spectrograms_frame_negative_support_undersampling_episodes, file_name)
    if os.path.exists(file_path):
       continue

    else:
        episodi_non_presenti.append(j)
print(episodi_non_presenti)


codice_ritorno= training_directory_path_spectrograms_frame_negative_support_undersampling_episodes(labels,train_file_paths,support_set,episodi_non_presenti=episodi_non_presenti, num_support_per_class=5)
if(codice_ritorno==1):
    print("Creazione conclusa")

### Creation with spectrograms created for mean positive events duration and not for the single frame.

In [141]:
episodi_non_presenti=[]
for j in range(1,55):
    file_name=f'training_episode {j}.pth'
    file_path=os.path.join(training_directory_path_spectrograms_episodes, file_name)
    if os.path.exists(file_path):
       continue

    else:
        episodi_non_presenti.append(j)
print(episodi_non_presenti)


codice_ritorno= create_support_and_query_sets_with_new_spectrogram_creation(labels,train_file_paths,support_set,episodi_non_presenti=episodi_non_presenti, num_support_per_class=5)
if(codice_ritorno==1):
    print("Creazione conclusa")

[1, 2]
 Inizio creazione support e query per l'audio: e1.wav
360000
184333
208173
Finito episodio,salvataggio con successo: 1
360000
205965
163770
Finito episodio,salvataggio con successo: 2
 Episodio 3 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: h1.wav
 Episodio 4 già presente, si passa al successivo
 Episodio 5 già presente, si passa al successivo
 Episodio 6 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: y1.wav
 Episodio 7 già presente, si passa al successivo
 Episodio 8 già presente, si passa al successivo
 Episodio 9 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: n1.wav
 Episodio 10 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: a1.wav
 Episodio 11 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: sp4f1_2015_04_24-05_00_00_0002.wav
 Episodio 12 già presente, si passa al successivo
 Inizio creazione 

### Spectrogram Creation for AST with 16Khz

In [36]:
episodi_non_presenti=[]
for j in range(1,55):
    file_name=f'training_episode {j}.pth'
    file_path=os.path.join(training_directory_path_spectrograms_episodes_AST, file_name)
    if os.path.exists(file_path):
       continue

    else:
        episodi_non_presenti.append(j)
print(episodi_non_presenti)


codice_ritorno= create_support_and_query_sets_with_new_spectrogram_creationAST_16KHz(labels,train_file_paths,support_set,episodi_non_presenti=episodi_non_presenti, num_support_per_class=5)
if(codice_ritorno==1):
    print("Creazione conclusa")

[37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]
 Inizio creazione support e query per l'audio: e1.wav
 Episodio 1 già presente, si passa al successivo
 Episodio 2 già presente, si passa al successivo
 Episodio 3 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: h1.wav
 Episodio 4 già presente, si passa al successivo
 Episodio 5 già presente, si passa al successivo
 Episodio 6 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: y1.wav
 Episodio 7 già presente, si passa al successivo
 Episodio 8 già presente, si passa al successivo
 Episodio 9 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: n1.wav
 Episodio 10 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: a1.wav
 Episodio 11 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: sp4f1_2015_04_24-05_00_00_0002.wav
 Episodio 12 già presente, si pass

C:\Users\aldob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


720000
11758
518606
Finito episodio,salvataggio con successo: 37
720000
12322
517792
Finito episodio,salvataggio con successo: 38
720000
8894
521213
Finito episodio,salvataggio con successo: 39
720000
147021
383064
Finito episodio,salvataggio con successo: 40
720000
51935
478170
Finito episodio,salvataggio con successo: 41
720000
419946
110143
Finito episodio,salvataggio con successo: 42
720000
86851
443289
Finito episodio,salvataggio con successo: 43
 Inizio creazione support e query per l'audio: 2015-10-14_23-59-59_unit05.wav
720000
33003
497123
Finito episodio,salvataggio con successo: 44
720000
247276
282865
Finito episodio,salvataggio con successo: 45
720000
52717
477411
Finito episodio,salvataggio con successo: 46
720000
75360
454817
Finito episodio,salvataggio con successo: 47
720000
33539
496562
Finito episodio,salvataggio con successo: 48
 Inizio creazione support e query per l'audio: 2015-09-11_06-00-00_unit07.wav
720000
43471
486684
Finito episodio,salvataggio con successo: 

### Support and query Set creation for raw audio + save

In [122]:
episodi_non_presenti=[]
for j in range(1,55):
    file_name=f'training_episode {j}.pth'
    file_path=os.path.join(training_directory_path_raw_audio_episodes, file_name)
    if os.path.exists(file_path):
       continue

    else:
        episodi_non_presenti.append(j)
print(episodi_non_presenti)



codice_ritorno=create_support_and_query_sets_raw_audio(labels,train_file_paths,support_set,episodi_non_presenti=episodi_non_presenti, num_support_per_class=5)
if(codice_ritorno==1):
    print("Creazione conclusa")

[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]
 Inizio creazione support e query per l'audio: e1.wav
 Episodio 1 già presente, si passa al successivo
 Episodio 2 già presente, si passa al successivo
 Episodio 3 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: h1.wav
 Episodio 4 già presente, si passa al successivo
 Episodio 5 già presente, si passa al successivo
 Episodio 6 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: y1.wav
 Episodio 7 già presente, si passa al successivo
 Episodio 8 già presente, si passa al successivo
 Episodio 9 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: n1.wav
 Episodio 10 già presente, si passa al successivo
 Inizio creazione support e query per l'audio: a1.wav
 Episodio 11 già presente, si passa al successivo
 Inizio creazion

C:\Users\aldob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1472
  warnings.warn(


40685
40687.43125
Finito episodio,salvataggio con successo: 12
 Inizio creazione support e query per l'audio: dcase_MK2.wav


C:\Users\aldob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1904
  warnings.warn(


259483
259485.2
Finito episodio,salvataggio con successo: 13
260696
260698.2
Finito episodio,salvataggio con successo: 14


...

125854
125855.9
Finito episodio,salvataggio con successo: 15
 Inizio creazione support e query per l'audio: dcase_MK1.wav


C:\Users\aldob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1552
  warnings.warn(
C:\Users\aldob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1440
  warnings.warn(
C:\Users\aldob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1200
  warnings.warn(


190525
190526.9
Finito episodio,salvataggio con successo: 16
89725
89726.90000000001
Finito episodio,salvataggio con successo: 17
31558
31559.900000000023
Finito episodio,salvataggio con successo: 18


C:\Users\aldob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=672
  warnings.warn(
C:\Users\aldob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=512
  warnings.warn(
C:\Users\aldob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=400
  warnings.warn(
C:\Users\aldob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=688
 

101011
101012.1
Finito episodio,salvataggio con successo: 19
 Inizio creazione support e query per l'audio: 2015-09-25_04-00-00_unit10.wav
582115
582116.1
Finito episodio,salvataggio con successo: 20
225311
225312.90000000002
Finito episodio,salvataggio con successo: 21
697116
697117.7
Finito episodio,salvataggio con successo: 22
569208
569209.2
Finito episodio,salvataggio con successo: 23
678394
678395.1
Finito episodio,salvataggio con successo: 24
717063
717064.1
Finito episodio,salvataggio con successo: 25
440775
440776.7
Finito episodio,salvataggio con successo: 26
712542
712543.8
Finito episodio,salvataggio con successo: 27
637579
637580.6
Finito episodio,salvataggio con successo: 28
 Inizio creazione support e query per l'audio: 2015-09-21_06-00-00_unit05.wav
133845
133846.0
Finito episodio,salvataggio con successo: 29
598124
598125.3
Finito episodio,salvataggio con successo: 30
588027
588028.3
Finito episodio,salvataggio con successo: 31
537143
537144.3
Finito episodio,salvatagg

# Prototypical Network

## From scratch trained convolutional network encoder  definition

In [72]:
#Notes: using spectrograms of dimension 128*2 the max pooling is done only to the height, to avoid the dimension to become 0
class PrototypicalNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(PrototypicalNetwork, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),
            nn.Flatten(),
            nn.Linear(256 *int(input_dim[0]/8)*input_dim[1], hidden_dim),
            nn.ReLU(),

        )
       

        # Classification head used for the auxiliary task of multiclass classification.
        self.task_auxiliary_head = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embeddings = self.encoder(x)
        task_auxiliary_output = self.task_auxiliary_head(embeddings)


        return embeddings,2, task_auxiliary_output


#function not used.
def get_audio_spectrogram(filepath, n_mels=64, n_fft=1024, hop_length=512):
    waveform, sample_rate = librosa.load(filepath, sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(
        y=waveform,
        sr=sample_rate,
        n_fft=n_fft,
        hop_length=hop_length,
        n_mels=n_mels
    )


    pcen_spectrogram = librosa.pcen(mel_spectrogram * (2**31))
    pcen_spectrogram = torch.tensor(pcen_spectrogram,dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions

    return pcen_spectrogram

In [73]:
class PrototypicalNetwork_with_binary_head(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(PrototypicalNetwork_with_binary_head, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),
            nn.Flatten(),
            nn.Linear(256 *int(input_dim[0]/8)*input_dim[1], hidden_dim),
            nn.ReLU(),

        )
       

        # Classification head used for the auxiliary task of multiclass classification.
        self.task_auxiliary_head = nn.Linear(hidden_dim, output_dim)
        self.binary_head = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        embeddings = self.encoder(x)
        task_auxiliary_output = self.task_auxiliary_head(embeddings)
        binary_output = self.binary_head(embeddings)


        return embeddings,binary_output, task_auxiliary_output


#function not used.
def get_audio_spectrogram(filepath, n_mels=64, n_fft=1024, hop_length=512):
    waveform, sample_rate = librosa.load(filepath, sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(
        y=waveform,
        sr=sample_rate,
        n_fft=n_fft,
        hop_length=hop_length,
        n_mels=n_mels
    )


    pcen_spectrogram = librosa.pcen(mel_spectrogram * (2**31))
    pcen_spectrogram = torch.tensor(pcen_spectrogram,dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions

    return pcen_spectrogram

## AST Pre-trained encoder definition

In [74]:
class AST_Model(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        # load AST model
        self.encoder = ASTModel.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593",attn_implementation="sdpa")
        #self.encoder=ASTModel.from_pretrained("MIT/ast-finetuned-speech-commands-v2",attn_implementation="sdpa")  ## va scelto uno dei 2,proviamo speech commands.
        # hidden_size = depth of the model
        self.hidden_size = self.encoder.config.hidden_size
        # freeze ast parameters
        for n, p in self.encoder.named_parameters():
            p.requires_grad = False


        self.intermediate_layer1 = nn.Linear(768, 768*4)
        self.intermediate_layer2 = nn.Linear(768*4, 768)

  

        self.task_auxiliary_head = nn.Sequential(
            nn.Linear(self.hidden_size, 384),
            nn.Linear(384, output_dim)
        )
 



    def forward(self, x):
        
        embeddings = self.encoder(x)["last_hidden_state"][:, 0, :]
        embeddings = self.intermediate_layer1(embeddings)
        embeddings=self.intermediate_layer2(embeddings)


        
        task_auxiliary_output = self.task_auxiliary_head(embeddings)

        return embeddings,2,task_auxiliary_output


Spectrogram concatenation on the time-axis to get a spectrogram usable for AST. Default spectrogram shape of (128,100)

In [75]:
def ASTSpectrogramConcatenation(tensor,time_spectrogram_dimension=130):
    shape=tensor.shape
    print(shape)
    #tensor = torch.cat(tensor, dim=2)
    new_spectrograms_count = math.floor(shape[0] // math.floor(time_spectrogram_dimension//2))
    tensor=tensor[:new_spectrograms_count*int(time_spectrogram_dimension//2),  :]
    tensor=tensor.view(new_spectrograms_count, 128, time_spectrogram_dimension)
    return tensor

Label correction to obtain a suitable target for the query_set. If the spectrograms go from shape (x,128,2) to (x/50,128,100), the labels must be unified.


In [76]:
def ASTLabelCorrection(labels):
    new_labels=[]
    label_number=len(labels)
    new_labels_number=math.floor(label_number/65)
    labels=labels[:new_labels_number*65]

    for i in range(new_labels_number):
        start_index = i * 65
        end_index = start_index + 65
        block=labels[start_index:end_index]
        #Let's take the most frequent label value.
        count=Counter(block)
        print(count)
        most_frequent_value = count.most_common(1)[0][0]

        new_labels.append(most_frequent_value)

    return new_labels

## Wave2Vec Pre-trained encoder definition

In [77]:
class CustomWav2Vec2Model(nn.Module):
    def __init__(self, base_model_name, num_labels_task_b=22,config=None):
        super(CustomWav2Vec2Model, self).__init__()


        if config is None:
            self.wav2vec2_encoder = Wav2Vec2Model.from_pretrained(base_model_name, torch_dtype=torch.float16, attn_implementation="flash_attention_2",ignore_mismatched_sizes=True)
        else:
            self.wav2vec2_encoder = Wav2Vec2Model.from_pretrained(base_model_name, config=config, torch_dtype=torch.float16, attn_implementation="flash_attention_2",ignore_mismatched_sizes=True)
        for n, p in self.wav2vec2_encoder.named_parameters():
            p.requires_grad = False


        #intermediate layer to implement a learning process
        self.intermediate_layer = nn.Linear(self.wav2vec2_encoder.config.hidden_size, 768)
        # Classification head for the tasks.task b is  multiclass.
        self.classifier_task_b = nn.Linear(768, num_labels_task_b)

        


    def forward(self, input_values, attention_mask=None):

        outputs = self.wav2vec2_encoder(input_values, attention_mask=attention_mask)
        embeddings = outputs.last_hidden_state
        embeddings=self.intermediate_layer(embeddings)
        #pooled_output = hidden_states.mean(dim=1)  ##teoricamente embedding aggregato usando la media, non so se abbia senso usarlo

        multiclass_classification_task = self.classifier_task_b(embeddings)

        return embeddings,2, multiclass_classification_task

Time-domain waveform concatenation to have a suitable input for Wav2Vec2. It's required at least a tensor with shape (batch_size,1000). Every frame has a shape of (220). Therefore we will concatenate 5 frame.

In [78]:
def Wav2VecWaveformConcatenation(tensor,time_domain_waveform_dimension=3300):
    shape=tensor.shape
    #tensor = torch.cat(tensor, dim=2)
    new_waveform_count = math.floor(shape[0] // math.floor(time_domain_waveform_dimension//220))
    tensor=tensor[:new_waveform_count*int(time_domain_waveform_dimension//220),:]
    tensor=tensor.view(new_waveform_count, time_domain_waveform_dimension)
    return tensor

Label correction for Wav2Vec. 5 frames are concatenated, therefore the corrispective frames must be aggregated

In [79]:
def Wav2VecLabelCorrection(labels,time_domain_waveform_dimension=3300):
    n_times=int(time_domain_waveform_dimension/220)
    new_labels=[]
    label_number=len(labels)
    new_labels_number=math.floor(label_number/n_times)
    labels=labels[:new_labels_number*n_times]

    for i in range(new_labels_number):
        start_index = i * n_times
        end_index = start_index + n_times
        block=labels[start_index:end_index]
        #Let's take the most frequent label value.
        count=Counter(block)
        most_frequent_value = count.most_common(1)[0][0]

        new_labels.append(most_frequent_value)

    return new_labels

# Cleanup phase

In [33]:
labels=0
ast_model=0
Wav2Vec_model=0
output=0
dataset_huggingface=0
inputs=0
feature_extractor=0
spectrogram=0


# Training phase

## Training utilities


In [80]:
def compute_class_weights(y):

    class_counts = Counter(y)

    total_samples = len(y)


    class_weights = {}
    for cls, count in class_counts.items():
        weight = total_samples / (len(class_counts) * count)
        class_weights[cls] = weight

    return class_weights

In [81]:
def query_set_majority_class_undersampling(query_samples,query_labels_binary_classification,query_labels):
    query_labels_binary_classification=query_labels_binary_classification.cpu().detach().numpy()
    query_labels=query_labels.cpu().detach().numpy()

    num_zeros = np.sum(query_labels_binary_classification == 0)
    num_ones = np.sum(query_labels_binary_classification == 1)
    minority_class_count = min(num_zeros, num_ones)

    zero_indices = np.where(query_labels_binary_classification == 0)[0]
    one_indices = np.where(query_labels_binary_classification == 1)[0]
    selected_zero_indices = np.random.choice(zero_indices, size=minority_class_count, replace=False)
    selected_one_indices = np.random.choice(one_indices, size=minority_class_count, replace=False)
    balanced_indices = np.concatenate([selected_zero_indices, selected_one_indices])

    balanced_query_samples = query_samples[balanced_indices]
    balanced_query_labels_binary_classification = query_labels_binary_classification[balanced_indices]
    query_labels=query_labels[balanced_indices]


    balanced_query_samples=torch.tensor(balanced_query_samples)
    balanced_query_labels_binary_classification=torch.tensor(balanced_query_labels_binary_classification)
    query_labels=torch.tensor(query_labels)

    return balanced_query_samples,balanced_query_labels_binary_classification,query_labels

## Training function 1.0 definition, for all the various encoder at once

In [43]:
def train_prototypical_network(model, dataloader, optimizer,  num_way, device,x=0,model_checkpoint=0,mode="classical_network",preprocessing="both",batch_size=32,class_unbalance="undersampling",rolling_mean=False):
    model.train()

    stats_dictionary={}

    #let's verify if there is an already loaded checkpoint for the corrisponding encoder. In the checkpoint will be even the last episode done,
    #so we can start from the next one skipping the already done ones.
    if mode=="AST":
        batch_size=512

    if(isinstance(model_checkpoint,int)):
        episode_done=0
    else:
        if x==1:
            path=os.path.join(checkpoint_path, "model_type_" + str(mode) +"_balanced_support_set.pth")
            if(os.path.exists(path)):
                stats_dictionary=torch.load(path)
            episode_done=model_checkpoint['last_episode_done']
        elif x==2:
            path=os.path.join(checkpoint_path, "model_type_" + str(mode) +"_complete_support_set.pth")
            if(os.path.exists(path)):
                stats_dictionary=torch.load(path)
            episode_done=model_checkpoint['last_episode_done']
        elif x==3:
            path=os.path.join(checkpoint_path, "model_type_" + str(mode) +"_event_spectrogram_set.pth")
            if(os.path.exists(path)):
                stats_dictionary=torch.load(path)
            episode_done=model_checkpoint['last_episode_done']
        else: 
            path=os.path.join(checkpoint_path, "model_type_" + str(mode)+".pth")
            if(os.path.exists(path)):
                stats_dictionary=torch.load(path)
            episode_done=model_checkpoint['last_episode_done']

    #i is the number of the current episode
    i=0

    j=0
    #j represents the number of total episode, here we got the number of episodes files in the directory.
    for file_name in dataloader:
        j=j+1

    #for each episode file it will train the model
    for file_name in dataloader:
        #it skips all the episodes beneath the last episode trained.
        if (i<episode_done):
            i=i+1
            print(f'Episode {i} already trained')
            continue
        #load model checkpoint if There is an already partially trained.
        if(model_checkpoint!=0):
            model.load_state_dict(model_checkpoint['model_state_dict'])
            optimizer.load_state_dict(model_checkpoint['optimizer_state_dict'])

        #based on the model type this notebook create a different directory with spectrograms or just raw audio. Based on the model requirements this will choose the directory to use
    
        if x==1:
            file_path=os.path.join(training_directory_path_spectrograms_frame_negative_support_undersampling_episodes, file_name)
        elif x==2:
            file_path=os.path.join(training_directory_path_spectrograms__all_negative_supports_episodes, file_name)
        else: 
            file_path=os.path.join(training_directory_path_spectrograms_episodes, file_name)


        if i==0:
            file_path=os.path.join(training_directory_path_spectrograms_episodes_AST, file_name)




        episode=torch.load(file_path)

        #from the already created episode now we will take the labels and the samples for query and support set.
        query_predictions=[]
    
        label_to_find_for_episode=episode[0]
        
        support_samples = [x[0] for x in episode[1]]
        support_labels = [x[1] for x in episode[1]]

        query_samples = [x[0] for x in episode[2]]
        query_labels =[x[1] for x in episode[2]]

        #query_labels for binary classification. In the audio if we are computing the F1 score we don't care about all the classes label, just the one to find.
        query_labels_binary_classification=[1 if element == label_to_find_for_episode else 0 for element in query_labels]

        count_of_positive_frames = sum(n == 1 for n in query_labels_binary_classification)
        
        if mode=="AST":
           
            if count_of_positive_frames<batch_size:
                print(i)
                i+=1
                continue

        if mode=="AST":
            
            support_samples=torch.tensor(support_samples)
            support_labels=torch.tensor(support_labels)
            print(support_samples.shape,support_labels.shape)
            mask_1 = support_labels == label_to_find_for_episode
            mask_0 = support_labels == 0    

            
            
            support_samples = torch.cat((support_samples[mask_1], support_samples[mask_0]), dim=0)
            support_labels = torch.cat((support_labels[mask_1], support_labels[mask_0]), dim=0)
            
            

        #when there are some del are just to avoid memory issues
        del episode

      


        #based on the network type the type of the samples tensor changes. Float32 for classical network and Half(Float16) for
        if(mode=="classical_network"):
            support_samples = torch.tensor(support_samples,dtype=torch.float32)#.to(device)
            query_samples = torch.tensor(query_samples,dtype=torch.float32)#.to(device)
        else:
          support_samples = torch.tensor(support_samples,dtype=torch.float16).to(device)
          query_samples = torch.tensor(query_samples,dtype=torch.float16).to(device)


        #data normalization/standardization/both
        ##Va visto se quando uso AST devo standardizzare rispetto al loro dataset e non al mio
        if(preprocessing=="normalization"):
            min_val = support_samples.min()
            max_val = support_samples.max()
            support_samples = (support_samples - min_val) / (max_val - min_val)
            query_samples = (query_samples - min_val) / (max_val - min_val)
        elif(preprocessing=="standardization"):
            mean = support_samples.mean()
            std = support_samples.std()
            support_samples = (support_samples - mean) / std
            query_samples = (query_samples - mean) / std
        elif(preprocessing=="both"):
            mean = support_samples.mean()
            std = support_samples.std()
            support_samples = (support_samples - mean) / std
            query_samples = (query_samples - mean) / std

            min_val = support_samples.min()
            max_val = support_samples.max()
            support_samples = (support_samples - min_val) / (max_val - min_val)
            query_samples = (query_samples - min_val) / (max_val - min_val)

           


        # samples reshaping to fit the different necessary input shape for the 3 different models.
        if(mode=="AST"):
            feature_extractor = AutoFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
            
            support_samples=support_samples.view(-1, 1, 128, 2)
            query_samples=query_samples.view(-1, 1, 128, 2)
            """
            query_samples=ASTSpectrogramConcatenation(query_samples)
            query_labels=ASTLabelCorrection(query_labels)
            query_labels_binary_classification=ASTLabelCorrection(query_labels_binary_classification)

            support_samples=ASTSpectrogramConcatenation(support_samples)
            support_labels=ASTLabelCorrection(support_labels)
            """
        elif(mode=="classical_network"):
            
            support_samples=support_samples.view(-1, 1, 128, 2)
            query_samples=query_samples.view(-1, 1, 128, 2)
     


        #tensor conversion + to device for the samples and labels.
        support_labels = torch.tensor(support_labels)#.to(device)
        query_labels = torch.tensor(query_labels)#.to(device)
        query_labels_binary_classification=torch.tensor(query_labels_binary_classification).to(device)
        


        """
        print(support_samples.shape)
        print(support_labels.shape)
         
        print(query_samples.shape)
        print(query_labels.shape)
        """
        #there is a big problem with class imbalance(in the first episode 2k to 205k), it must be handled in some way.
        # 3 ways to handle this brutal situation.


        #1 Applying SMOTE directly. Then it will balance the classes sample. I'll do it anyway but in my opinion is really bad, overfitting assured...

        if class_unbalance=="SMOTE":
            #smote strategy =auto therefore it increase the minority class to reach the number of majority class.
            smote = SMOTE(sampling_strategy='auto', random_state=13)
            balanced_query_samples, balanced_query_labels_binary_classification = smote.fit_resample(query_samples, query_labels_binary_classification)
            query_labels_binary_classification=torch.tensor(query_labels_binary_classification,dtype=torch.long).to(device)

        elif class_unbalance=="undersampling":
            query_samples,query_labels_binary_classification,query_labels=query_set_majority_class_undersampling(query_samples,query_labels_binary_classification,query_labels)
            query_labels_binary_classification=torch.tensor(query_labels_binary_classification,dtype=torch.long)
            query_labels=torch.tensor(query_labels)

        elif class_unbalance=="both":
            query_labels_binary_classification=query_labels_binary_classification.cpu().detach().numpy()
            num_zeros = np.sum(query_labels_binary_classification == 0)
            num_ones = np.sum(query_labels_binary_classification == 1)
            minority_class_count = min(num_zeros, num_ones)

            minority_class = 0 if num_zeros < num_ones else 1
            minority_samples = query_samples[query_labels_binary_classification == minority_class]
            minority_labels = query_labels_binary_classification[query_labels_binary_classification == minority_class]


            #smote strategy =1.5 therefore it increase by 50% the number of samples.
            smote = SMOTE(sampling_strategy='1.5', random_state=13)
            minority_samples, minority_labels = smote.fit_resample(minority_samples, minority_labels)

            minority_class_count=len(minority_labels)
            ############àTo be continued
            zero_indices = np.where(query_labels_binary_classification == 0)[0]
            one_indices = np.where(query_labels_binary_classification == 1)[0]
            selected_zero_indices = np.random.choice(zero_indices, size=minority_class_count, replace=False)
            selected_one_indices = np.random.choice(one_indices, size=minority_class_count, replace=False)
            balanced_indices = np.concatenate([selected_zero_indices, selected_one_indices])

            balanced_query_samples = query_samples[balanced_indices]
            balanced_query_labels_binary_classification = query_labels_binary_classification[balanced_indices]

            query_labels_binary_classification=torch.tensor(query_labels_binary_classification,dtype=torch.long).to(device)
        else:
            pass


       
        query_samples_size=query_samples.shape[0]

        #creation of the Dataset using the tensor created before
        support_dataset = TensorDataset(support_samples, support_labels)
        query_dataset = TensorDataset(query_samples, query_labels, query_labels_binary_classification)

        del support_samples
        del support_labels
        del query_samples
        

        #loss function definition
        
        triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2) #, eps=1e-7?
        

        # weight for the amount of labels in the batch.
        count=Counter(query_labels.tolist())

        print(count)
        weight_dict=compute_class_weights(query_labels.tolist())
        weight_vector = [0.0] * num_way
        for classs, weight in weight_dict.items():
            weight_vector[classs] = weight
        class_weights = torch.tensor(weight_vector,dtype=torch.float32)
        cross_entropy_weighted = nn.CrossEntropyLoss(weight=class_weights) 
        cross_entropy_weighted=cross_entropy_weighted

        cross_entropy=nn.CrossEntropyLoss()



        #loader for support and query set for the current episode. Shuffle=True for classical network, no shuffle for every type of encoders.
       
        support_loader = DataLoader(support_dataset, batch_size=batch_size, shuffle=False)
        query_loader = DataLoader(query_dataset, batch_size=batch_size, shuffle=False)

      
        # Prototypes creation for all the support set for the episode.

        #will be used 2 tensor, 1 for the positive and 1 for the negative prototype. For memory issue the prototype will be computed for each batch and will be made
        #the mean with the prototype for the same class of the subsequent batch.
        
        aggregated_positive_prototype=None
        aggregated_negative_prototype=None
        with torch.no_grad():
            if rolling_mean==False:
                negative_embeddings=[]
                positive_embeddings=[]
            for support_batch,support_labels_batch in support_loader:

                if rolling_mean==True:
                    negative_embeddings=[]
                    positive_embeddings=[]
                
                support_batch = support_batch.to(device)
                support_labels_batch=support_labels_batch.to(device)
                
                
                
                if mode=="classical_network":
                    with autocast():
                        for x in range(0,support_batch.shape[0]):
                            
                            frame=support_batch[x].unsqueeze(0)
                            support_embeddings, _, _= model(frame)
                            torch.cuda.empty_cache()
                            if (support_labels_batch[x]==0):
                                negative_embeddings.append(support_embeddings)
                            else:
                                positive_embeddings.append(support_embeddings)
                elif mode=="AST":
                    if support_batch.shape[0]<batch_size:
                        continue
                    support_batch=support_batch.squeeze(dim=1)
                    support_batch = support_batch.permute(0, 2, 1)
                    support_batch = support_batch.reshape(-1, 1024)
                    support_batch = support_batch.unsqueeze(0)
                    with autocast():
                        most_frequent_label = np.bincount(support_labels_batch.cpu()).argmax()
                        batch_embedding,_,_=model(support_batch)

                        if (most_frequent_label==0):
                            negative_embeddings.append(batch_embedding)
                        else:
                            positive_embeddings.append(batch_embedding)
                if rolling_mean==True:
                    if len(positive_embeddings) > 0:
                        positive_embeddings=torch.stack(positive_embeddings)
                        prototype = positive_embeddings.mean(dim=0)
                        
                        if aggregated_positive_prototype is None:
                            aggregated_positive_prototype=prototype
                        else:
                            aggregated_positive_prototype = (torch.stack([aggregated_positive_prototype, prototype])).mean(dim=0)

                    if len(negative_embeddings) > 0:
                        negative_embeddings=torch.stack(negative_embeddings)
                        prototype = negative_embeddings.mean(dim=0)
                        if aggregated_negative_prototype is None:
                            aggregated_negative_prototype=prototype
                        else:
                            aggregated_negative_prototype = (torch.stack([aggregated_negative_prototype, prototype])).mean(dim=0)


            if rolling_mean==False:
                if len(positive_embeddings) > 0:
                    positive_embeddings=torch.stack(positive_embeddings)
                    prototype = positive_embeddings.mean(dim=0)
                    
                    if aggregated_positive_prototype is None:
                        aggregated_positive_prototype=prototype
                    else:
                        aggregated_positive_prototype = (torch.stack([aggregated_positive_prototype, prototype])).mean(dim=0)

                if len(negative_embeddings) > 0:
                    negative_embeddings=torch.stack(negative_embeddings)
                    prototype = negative_embeddings.mean(dim=0)
                    if aggregated_negative_prototype is None:
                        aggregated_negative_prototype=prototype
                    else:
                        aggregated_negative_prototype = (torch.stack([aggregated_negative_prototype, prototype])).mean(dim=0)


            torch.cuda.empty_cache()
            negative_embeddings=[]
            positive_embeddings=[]
                
        #adding the negative and positive prototype for the entire episode.
        negative_prototype=aggregated_negative_prototype
        positive_prototype=aggregated_positive_prototype
        
        del support_batch
        del support_labels_batch
    
        torch.cuda.empty_cache()

        #starting the query set phase. In this it will be trained the model based on the 2 loss function and then calculated the F1 score for the predictions done based on the prototypes created before.
        progress_bar = tqdm(total=query_samples_size)
        
        episode_loss=0
        for query_batch, query_labels_batch, query_labels_binary_classification_batch in query_loader:


            query_batch = query_batch.to(device)
            query_labels_batch = query_labels_batch.to(device)
            query_labels_binary_classification_batch = query_labels_binary_classification_batch.to(device)

            cumulative_loss=0
            optimizer.zero_grad()

            if mode=="AST":
                with autocast():
                    if query_batch.shape[0]<batch_size:
                        continue
                    query_batch=query_batch.squeeze(dim=1)
                    query_batch = query_batch.permute(0, 2, 1)
                    query_batch = query_batch.reshape(-1, 1024)
                    query_batch = query_batch.unsqueeze(0)
                    with autocast():
                        most_frequent_label = np.bincount(query_labels_batch.cpu()).argmax()
                        most_frequent_binary_label=np.bincount(query_labels_batch.cpu()).argmax()

                        query_embeddings,_,multiclass_query_output = model(query_batch)
                        most_frequent_label=torch.tensor(most_frequent_label)
                        most_frequent_label=most_frequent_label.unsqueeze(0).to(device)
                        auxiliary_task_loss=cross_entropy(multiclass_query_output, most_frequent_label)
                        
                        if(most_frequent_binary_label==0):
                            binary_loss = triplet_loss(query_embeddings, negative_prototype,positive_prototype)
                        else:
                            binary_loss = triplet_loss(query_embeddings, positive_prototype,negative_prototype)

        
                        loss = binary_loss + 0.8*auxiliary_task_loss 
                            
                        query_embeddings.detach()
                        multiclass_query_output.detach()

                        if(cumulative_loss==0):
                            cumulative_loss=loss
                        else: 
                            cumulative_loss+=loss

                        if(episode_loss==0):
                            episode_loss=loss
                        else:
                            episode_loss+=loss
                        with torch.no_grad():
                            distance_from_negative = F.pairwise_distance(query_embeddings,negative_prototype)
                            distance_from_positive=F.pairwise_distance(query_embeddings, positive_prototype)
                        if(distance_from_positive<distance_from_negative):
                            for _ in range(0,batch_size):
                                query_predictions.append(1)
                        else:
                            for _ in range(0,batch_size):
                                query_predictions.append(0)

                        progress_bar.update(batch_size)

                        
            elif mode=="classical_network":
                for y in range(0,query_batch.shape[0]):
                    
                    #with autocast():
                        
                    frame=query_batch[y]
                    frame=frame.unsqueeze(0)
                    query_embeddings, _, multiclass_query_output = model(frame)

                    """"
                    print(f'support_embeddings :{support_embeddings.shape}')
                    print(f'binary_support_output :{binary_support_output.shape}')
                    print(f'multiclass_support_output :{multiclass_support_output.shape}')
                    print(f'query_embeddings :{query_embeddings.shape}')
                    print(f'binary_query_output :{binary_query_output.shape}')
                    print(f'multiclass_query_output :{multiclass_query_output.shape}')
                    

                    #let's replicate the label value for the sample for each 10 token created by the model.
                    
                    if (mode=="Wav2Vec"):
                        print(multiclass_query_output.shape)
                        multiclass_query_output = multiclass_query_output.permute(0, 2, 1)
                        print(multiclass_query_output.shape)
                    
                        print(query_labels_batch.shape)
                        query_labels_batch = query_labels_batch.unsqueeze(1).expand(-1, 10)
                    """


                    multiclass_query_output=multiclass_query_output.float()
                    
                    
                    with autocast():
                        frame_label=query_labels_batch[y]
                        if mode=="classical_network":
                            frame_label=frame_label.unsqueeze(0)
                        if(class_unbalance=="undersampling"):
                            auxiliary_task_loss=cross_entropy(multiclass_query_output, frame_label)
                        else:
                            auxiliary_task_loss = cross_entropy_weighted(multiclass_query_output, frame_label)

                    
                    if(query_labels_binary_classification_batch[y]==0):
                        binary_loss = triplet_loss(query_embeddings, negative_prototype,positive_prototype)
                    else:
                        binary_loss = triplet_loss(query_embeddings, positive_prototype,negative_prototype)

                    
                    loss = binary_loss + 0.8*auxiliary_task_loss  ## va scelto il coefficente dell'auxiliary task.
                    
                    query_embeddings.detach()
                    multiclass_query_output.detach()

                    if(cumulative_loss==0):
                        cumulative_loss=loss
                    else: 
                        cumulative_loss+=loss

                    if(episode_loss==0):
                        episode_loss=loss
                    else:
                        episode_loss+=loss
                    
                    with torch.no_grad():
                        distance_from_negative = F.pairwise_distance(query_embeddings,negative_prototype)
                        distance_from_positive=F.pairwise_distance(query_embeddings, positive_prototype)
                    
                    if(mode=="Wav2Vec"):
                        
                        distance_from_negative=distance_from_negative.squeeze()
                        distance_from_positive=distance_from_positive.squeeze()
                        distance_from_negative=distance_from_negative.mean(dim=0)
                        distance_from_positive=distance_from_positive.mean(dim=0)
                    
                    
                    if(distance_from_positive<distance_from_negative):
                        query_predictions.append(1)
                    else:
                        query_predictions.append(0)

                    progress_bar.update(1)
                    
                    
                    del query_embeddings
                    del multiclass_query_output
                    del distance_from_negative
                    del distance_from_positive
            torch.cuda.empty_cache()
            

            if mode=="AST":
                mean_loss=cumulative_loss
            elif mode=="classical_network":
                mean_loss=cumulative_loss/query_batch.shape[0]
            
            mean_loss.backward()
            optimizer.step()
            
            del query_batch
            del query_labels_batch
            del query_labels_binary_classification_batch


        if mode=="AST":episode_loss=episode_loss*batch_size/query_labels.shape[0]
        elif mode=="classical_network":
            episode_loss=episode_loss/query_labels.shape[0]
        #Code to verify an eventual unbalancing of the classes(spoiler 2k to 205k in episode 1) SMOTE + data augmentation...
        query_predictions_np = np.array(query_predictions)
        count_of_ones = np.sum(query_predictions_np)
        count_of_ones_2=np.sum(query_labels_binary_classification.cpu().detach().numpy())
        

        print(f'Numero frame classificate positive:{count_of_ones},Numero frame realmente positive:{count_of_ones_2}')
        

        #Let's compute the F1 score for the whole episode.
        if len(query_predictions_np)<len(query_labels_binary_classification.cpu().detach().numpy()):
            query_labels_binary_classification=query_labels_binary_classification[:len(query_predictions_np)]
            
        f1 = f1_score(query_labels_binary_classification.cpu().detach().numpy(), query_predictions_np, average='binary')
        precision=precision_score(query_labels_binary_classification.cpu().detach().numpy(), query_predictions_np)
        recall=recall_score(query_labels_binary_classification.cpu().detach().numpy(), query_predictions_np)
        cm = confusion_matrix(query_labels_binary_classification.cpu().detach().numpy(), query_predictions_np)
        


        print(f'Episode {i + 1}/{j}, Episode Mean Loss: {episode_loss.item()}')
        print(f'Episode {i + 1}/{j}, F1_Score: {f1}')
        print(f'Episode {i + 1}/{j}, Precision: {precision}')
        print(f'Episode {i + 1}/{j}, Recall: {recall}')

            
        if(class_unbalance=="undersampling"):
            sampling="_undersampled"
        else:
            sampling=""
        dictionary={}
        dictionary["F1_Score"]=f1
        dictionary["Precision"]=precision
        dictionary["Recall"]=recall
        dictionary["Confusion_Matrix"]=cm
        
        string="Episode: "+str(i+1)
        stats_dictionary[string]=dictionary
        for param_group in optimizer.param_groups:
            current_lr = param_group['lr']


        #saving stats

        if not os.path.exists(directory_path_training_stats):
            os.makedirs(directory_path_training_stats)
        torch.save({'stats_dict':stats_dictionary,
                    'learning rate':current_lr,
                    'class_distribution_query_set':count
        },os.path.join(directory_path_training_stats, "training_stats_" + str(mode) +str(x)+ ".pth"))



        #saving the model parameters based on the encoder type.

        if rolling_mean==True:
            rolling="_rolling_"
        else:
            rolling=""
        
        if mode=="AST" or mode=="Wav2Vec":
            os.makedirs(checkpoint_path, exist_ok=True)
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'last_episode_done':i+1
            }, os.path.join(checkpoint_path, "model_type_" + str(mode)+str(rolling) +".pth"))
            
        else: 
            if x==1:
                os.makedirs(checkpoint_path, exist_ok=True)
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'last_episode_done':i+1
                }, os.path.join(checkpoint_path, "model_type_" + str(mode)+str(rolling)+"_balanced_support_set.pth"))
            elif x==2:
                os.makedirs(checkpoint_path, exist_ok=True)
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'last_episode_done':i+1
                }, os.path.join(checkpoint_path, "model_type_" + str(mode)+str(rolling)+"_complete_support_set.pth"))
            else: 
                os.makedirs(checkpoint_path, exist_ok=True)
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'last_episode_done':i+1
                }, os.path.join(checkpoint_path, "model_type_" + str(mode)+str(rolling)+"_event_spectrogram.pth"))
        
            
        i=i+1


## Training procedure for Wav2Vec

In [44]:
def train_prototypical_network_Wav2Vec(model, dataloader, optimizer,  num_way, device,x=0,model_checkpoint=0,mode="Wav2Vec",preprocessing="both",batch_size=32,class_unbalance="undersampling"):
    model.train()
    model=model.to(device)
    stats_dictionary={}
    if(isinstance(model_checkpoint,int)):
        episode_done=0
    else:
        if x==1:
            path=os.path.join(checkpoint_path, "model_type_" + str(mode) +"_balanced_support_set.pth")
            if(os.path.exists(path)):
                stats_dictionary=torch.load(path)
            episode_done=model_checkpoint['last_episode_done']
        elif x==2:
            path=os.path.join(checkpoint_path, "model_type_" + str(mode) +"_complete_support_set.pth")
            if(os.path.exists(path)):
                stats_dictionary=torch.load(path)
            episode_done=model_checkpoint['last_episode_done']
        elif x==3:
            path=os.path.join(checkpoint_path, "model_type_" + str(mode) +"_event_spectrogram_set.pth")
            if(os.path.exists(path)):
                stats_dictionary=torch.load(path)
            episode_done=model_checkpoint['last_episode_done']
        else:
            path=os.path.join(checkpoint_path, "model_type_" + str(mode)+".pth")
            if(os.path.exists(path)):
                stats_dictionary=torch.load(path)
            episode_done=model_checkpoint['last_episode_done']

    #i is the number of the current episode
    i=0

    j=0
    for file_name in dataloader:
        j=j+1

    #for each episode file it will train the model
    for file_name in dataloader:
        #it skips all the episodes beneath the last episode trained.
        if (i<episode_done):
            i=i+1
            print(f'Episode {i} already trained')
            continue
        #load model checkpoint if There is an already partially trained.
        if(model_checkpoint!=0):
            model.load_state_dict(model_checkpoint['model_state_dict'])
            optimizer.load_state_dict(model_checkpoint['optimizer_state_dict'])

        file_path=os.path.join(training_directory_path_raw_audio_episodes, file_name)
        episode=torch.load(file_path)

        query_predictions=[]

        label_to_find_for_episode=episode[0]

        support_samples = [x[0] for x in episode[1]]
        support_labels = [x[1] for x in episode[1]]

        query_samples = [x[0] for x in episode[2]]
        query_labels =[x[1] for x in episode[2]]


        support_samples=torch.tensor(support_samples)
        support_labels=torch.tensor(support_labels)
        print(support_samples.shape,support_labels.shape)
      
        mask_1 = support_labels == label_to_find_for_episode
        mask_0 = support_labels == 0



        support_samples = torch.cat((support_samples[mask_1], support_samples[mask_0]), dim=0)
        support_labels = torch.cat((support_labels[mask_1], support_labels[mask_0]), dim=0)

        processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
        processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")


        del episode
        query_labels_binary_classification=[1 if element == label_to_find_for_episode else 0 for element in query_labels]

        count_of_ones = np.sum(query_labels_binary_classification)
        if(count_of_ones)<batch_size:
            print(i)
            i+=1
            continue

        support_samples = torch.tensor(support_samples,dtype=torch.float16).to(device)
        query_samples = torch.tensor(query_samples,dtype=torch.float16).to(device)
       
        mean = support_samples.mean()
        std = support_samples.std()
        support_samples = (support_samples - mean) / std
        query_samples = (query_samples - mean) / std

        min_val = support_samples.min()
        max_val = support_samples.max()
        support_samples = (support_samples - min_val) / (max_val - min_val)
        query_samples = (query_samples - min_val) / (max_val - min_val)
        
        processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")


        support_labels = torch.tensor(support_labels)#.to(device)
        query_labels = torch.tensor(query_labels)#.to(device)
        print(query_labels.shape)
        query_labels_binary_classification=torch.tensor(query_labels_binary_classification).to(device)

       
        query_samples,query_labels_binary_classification,query_labels=query_set_majority_class_undersampling(query_samples,query_labels_binary_classification,query_labels)
        query_labels_binary_classification=torch.tensor(query_labels_binary_classification,dtype=torch.long)
        query_labels=torch.tensor(query_labels)

        query_samples_size=query_samples.shape[0]

        support_dataset = TensorDataset(support_samples, support_labels)
        query_dataset = TensorDataset(query_samples, query_labels, query_labels_binary_classification)

        del support_samples
        del support_labels
        del query_samples

        triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2) #, eps=1e-7?

       
        # weight for the amount of labels in the batch.
        count=Counter(query_labels.tolist())

       
        weight_dict=compute_class_weights(query_labels.tolist())
        weight_vector = [0.0] * num_way
        for classs, weight in weight_dict.items():
            weight_vector[classs] = weight
        class_weights = torch.tensor(weight_vector,dtype=torch.float32)
        cross_entropy_weighted = nn.CrossEntropyLoss(weight=class_weights)
        cross_entropy_weighted=cross_entropy_weighted

        cross_entropy=nn.CrossEntropyLoss()

        support_loader = DataLoader(support_dataset, batch_size=batch_size, shuffle=False)
        query_loader = DataLoader(query_dataset, batch_size=batch_size, shuffle=False)


        aggregated_positive_prototype=None
        aggregated_negative_prototype=None
        negative_embeddings=[]
        positive_embeddings=[]

        with torch.no_grad():
                for support_batch,support_labels_batch in support_loader:
                    
                    
                    support_batch = support_batch
                    support_labels_batch=support_labels_batch
                    if support_batch.shape[0]<batch_size:
                        break
                    support_batch = support_batch.view(-1)
                    #the output of processor is a dictionay, using .input_values we are taking the right output.
                    support_batch=processor(support_batch, return_tensors="pt", sampling_rate=16000).input_values
                    support_batch=torch.tensor(support_batch,dtype=torch.half).to(device)
                
                    with autocast():
                        most_frequent_label = np.bincount(support_labels_batch.cpu()).argmax()
                        torch.cuda.empty_cache()
                        batch_embedding,_,_=model(support_batch)
                    
                    if (most_frequent_label==0):
                        negative_embeddings.append(batch_embedding)
                    else:
                        positive_embeddings.append(batch_embedding)
        
                if len(positive_embeddings) > 0:
                    positive_embeddings=torch.stack(positive_embeddings)
                    prototype = positive_embeddings.mean(dim=0)

                    if aggregated_positive_prototype is None:
                        aggregated_positive_prototype=prototype
                    else:
                        aggregated_positive_prototype = (torch.stack([aggregated_positive_prototype, prototype])).mean(dim=0)

                if len(negative_embeddings) > 0:
                    negative_embeddings=torch.stack(negative_embeddings)
                    prototype = negative_embeddings.mean(dim=0)
                    if aggregated_negative_prototype is None:
                        aggregated_negative_prototype=prototype
                    else:
                        aggregated_negative_prototype = (torch.stack([aggregated_negative_prototype, prototype])).mean(dim=0)


        torch.cuda.empty_cache()
        negative_embeddings=[]
        positive_embeddings=[]
        negative_prototype=aggregated_negative_prototype
        positive_prototype=aggregated_positive_prototype

        progress_bar = tqdm(total=query_samples_size)

        episode_loss=0
        for query_batch, query_labels_batch, query_labels_binary_classification_batch in query_loader:

            
            query_batch = query_batch.to(device)
            query_labels_batch = query_labels_batch.to(device)
            query_labels_binary_classification_batch = query_labels_binary_classification_batch.to(device)

            cumulative_loss=0
            if query_labels_batch.shape[0]<batch_size:
                break
            most_frequent_label = np.bincount(query_labels_batch.cpu()).argmax()
            most_frequent_binary_label=np.bincount(query_labels_batch.cpu()).argmax()

            query_batch = query_batch.view(-1)
        
            
            query_batch=processor(query_batch, return_tensors="pt", sampling_rate=16000).input_values

            query_batch=torch.tensor(query_batch,dtype=torch.half).to(device)

            with autocast():
                query_embeddings,_,multiclass_query_output = model(query_batch)

            aggregated_labels=[]
            l=0
            for h in range(0,32,2):
                if l==0:
                    if query_labels_batch[0]!= query_labels_batch[1]:
                        l+=1
                        continue

                if query_labels_batch[h]==query_labels_batch[h+1]:
                    aggregated_labels.append(query_labels_batch[h])
                else: 
                    if query_labels_batch[h]==0:
                        aggregated_labels.append(query_labels_batch[h+1])
                    else : 
                        aggregated_labels.append(query_labels_batch[h])
                    
            
            if len(aggregated_labels)>15:
                aggregated_labels=aggregated_labels[0:15]
                    
            aggregated_labels=torch.tensor(aggregated_labels).unsqueeze(0).to(device)
            
            multiclass_query_output=multiclass_query_output.permute(0,2,1)
            auxiliary_task_loss=cross_entropy(multiclass_query_output, aggregated_labels)
            
            if(most_frequent_binary_label==0):
                binary_loss = triplet_loss(query_embeddings, negative_prototype,positive_prototype)
            else:
                binary_loss = triplet_loss(query_embeddings, positive_prototype,negative_prototype)

            loss = binary_loss + 0.8*auxiliary_task_loss

            query_embeddings.detach()
            multiclass_query_output.detach()
            
            

            if(cumulative_loss==0):
                cumulative_loss=loss
            else:
                cumulative_loss+=loss

            if(episode_loss==0):
                episode_loss=loss
            else:
                episode_loss+=loss

            distance_from_negative = F.pairwise_distance(query_embeddings,negative_prototype)
            distance_from_positive=F.pairwise_distance(query_embeddings, positive_prototype)


            for h in range (0,15):
                if(distance_from_positive[0][h]<distance_from_negative[0][h]):
                        query_predictions.append(1)
                        query_predictions.append(1)
                else:
                        query_predictions.append(0)
                        query_predictions.append(0)
            
            query_predictions.append(query_predictions[-1])
            query_predictions.append(query_predictions[-1])
            
            progress_bar.update(batch_size)

            mean_loss=cumulative_loss

            optimizer.zero_grad()
            mean_loss.backward()
            optimizer.step()

            del query_batch
            del query_labels_batch
            del query_labels_binary_classification_batch

            episode_loss=episode_loss*batch_size/query_labels.shape[0]

        query_predictions_np = np.array(query_predictions)
        count_of_ones = np.sum(query_predictions_np)
        count_of_ones_2=np.sum(query_labels_binary_classification.cpu().detach().numpy())
        print(f'Predictate Positive:{count_of_ones}, Realmente Positive:{count_of_ones_2}')
      
        if len(query_predictions_np)<len(query_labels_binary_classification.cpu().detach().numpy()):
            query_labels_binary_classification=query_labels_binary_classification[:len(query_predictions_np)]

        f1 = f1_score(query_labels_binary_classification.cpu().detach().numpy(), query_predictions_np, average='binary')
        precision=precision_score(query_labels_binary_classification.cpu().detach().numpy(), query_predictions_np)
        recall=recall_score(query_labels_binary_classification.cpu().detach().numpy(), query_predictions_np)
        cm = confusion_matrix(query_labels_binary_classification.cpu().detach().numpy(), query_predictions_np)



        print(f'Episode {i + 1}/{j}, Episode Mean Loss: {episode_loss.item()}')
        print(f'Episode {i + 1}/{j}, F1_Score: {f1}')
        print(f'Episode {i + 1}/{j}, Precision: {precision}')
        print(f'Episode {i + 1}/{j}, Recall: {recall}')



        dictionary={}
        dictionary["F1_Score"]=f1
        dictionary["Precision"]=precision
        dictionary["Recall"]=recall
        dictionary["Confusion_Matrix"]=cm

        string="Episode: "+str(i+1)
        stats_dictionary[string]=dictionary
        for param_group in optimizer.param_groups:
            current_lr = param_group['lr']


        #saving stats

        if not os.path.exists(directory_path_training_stats):
            os.makedirs(directory_path_training_stats)
        torch.save({'stats_dict':stats_dictionary,
                    'learning rate':current_lr,
                    'class_distribution_query_set':count
        },os.path.join(directory_path_training_stats, "training_stats_" + str(mode) + ".pth"))



        #saving the model parameters based on the encoder type.

        os.makedirs(checkpoint_path, exist_ok=True)
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'last_episode_done':i+1
        }, os.path.join(checkpoint_path, "model_type_" + str(mode) +".pth"))
        i=i+1


## Running 1.0 version

In [82]:
#if i=1 then Wav2Vec, if i=0 AST. If i=2 CNN with negative support undersampling, if i=3 CNN with full negative support, if i=4 CNN with event spectrogram creation and full negative support.
i=3

rolling_mean=False


torch.cuda.empty_cache()
if (i==1):
  file_names=get_episodes_files_name(training_directory_path_raw_audio_episodes)
elif i==2:
  file_names=get_episodes_files_name(training_directory_path_spectrograms_frame_negative_support_undersampling_episodes)
elif i==3:
  file_names=get_episodes_files_name(training_directory_path_spectrograms__all_negative_supports_episodes)
elif i==4 :
  file_names=get_episodes_files_name(training_directory_path_spectrograms_episodes)
elif i==0:
   file_names=get_episodes_files_name(training_directory_path_spectrograms_episodes_AST)
else: 
   file_names=0
 

if i==0:
   """
   print(len(file_names))
   episodes_to_remove={9,10,11,13,18,21,28,39,45,49}
   file_names=[elemento for h,elemento in enumerate(file_names) if h not in episodes_to_remove]
   print(len(file_names))
   """
#num_way=len(label_mapping)
num_way=22
input_dim=0
input_dim = (128, 2)  # Dimension for the spectrogram in our cases.

hidden_dim = 768 #for the convolutional neural network encoder, choosen the same as AST and Wav2Vec
output_dim = num_way  # Dimensionality of the output space (prototype dimension)


CPU=False


if(CPU==True):
    device = torch.device("cpu")

else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


if i==1:
  """
  config = Wav2Vec2Config(
      sample_rate=22050,
      max_sample_size=220,
      min_sample_size=220
  )
  """
  Wav2Vec_model=0
  Wav2Vec_model = CustomWav2Vec2Model(
    base_model_name="facebook/wav2vec2-large-960h-lv60-self",
    num_labels_task_b=22,
    
).cuda()



if i==0:
  ast_model=0
  ast_model=AST_Model(output_dim).to(device)

if (i>1 or i<0):
  classic_model=0
  classic_model=PrototypicalNetwork(input_dim=input_dim,output_dim=output_dim,hidden_dim=hidden_dim).to(device)


if i==0:
  optimizer = torch.optim.Adam(ast_model.parameters(), lr=0.0001)
  mode="AST"
elif i==1:
  optimizer = torch.optim.Adam(Wav2Vec_model.parameters(), lr=0.001)
  mode="Wav2Vec"
else:
  optimizer = torch.optim.Adam(classic_model.parameters(), lr=0.001)
  mode="classical_network"

if rolling_mean==True:
    rolling="_rolling_"
else: 
    rolling=""


if i==0 or i==1:
  model_info=0
  x=0
  if os.path.isfile(os.path.join(checkpoint_path, "model_type_" + str(mode) +str(rolling)+".pth")):
      checkpoint_file=os.path.join(checkpoint_path, "model_type_" + str(mode) +str(rolling)+ ".pth")
      model_info=torch.load(checkpoint_file)
elif i==2:
    x=1
    model_info=0
    if os.path.isfile(os.path.join(checkpoint_path, "model_type_" + str(mode) +str(rolling)+"_balanced_support_set.pth")):
        checkpoint_file=os.path.join(checkpoint_path, "model_type_" + str(mode)+str(rolling)+ "_balanced_support_set.pth")
        model_info=torch.load(checkpoint_file)
elif i==3:
    x=2
    model_info=0
    if os.path.isfile(os.path.join(checkpoint_path, "model_type_" + str(mode) +str(rolling)+"_complete_support_set.pth")):
        checkpoint_file=os.path.join(checkpoint_path, "model_type_" + str(mode) +str(rolling)+ "_complete_support_set.pth")
        model_info=torch.load(checkpoint_file)
elif i==4:
    x=3
    model_info=0
    if os.path.isfile(os.path.join(checkpoint_path, "model_type_" + str(mode) +str(rolling)+ "_event_spectrogram.pth")):
        checkpoint_file=os.path.join(checkpoint_path, "model_type_" + str(mode) +str(rolling)+ "_event_spectrogram.pth")
        model_info=torch.load(checkpoint_file)
else: 
    model_info=0


if i==0:train_prototypical_network(ast_model, file_names, optimizer, num_way, device,mode=mode,model_checkpoint=model_info)
elif i==1:train_prototypical_network_Wav2Vec(Wav2Vec_model, file_names, optimizer, num_way, device,mode=mode,model_checkpoint=model_info)
else:train_prototypical_network(classic_model, file_names, optimizer, num_way, device,x=x,mode=mode,model_checkpoint=model_info)

KeyboardInterrupt: 

# Inference

## Support and Query Set Creation for just an audio

### Creation of support and query dictionaries containing positive events start and end

In [82]:
def get_support_set_inference(audio_name):
    audio_support_events=[]
    with open(val_json, 'r') as file:
        dati = json.load(file)
        for classe,value in dati.items():
            for csv in dati[classe]:
                if(dati[classe][csv]["Support"]!={}):
                    if dati[classe][csv]["Audio"] ==audio_name:

                        lista_start_e_end_time_eventi=[]
                        for evento in dati[classe][csv]["Support"]:
                            lista_start_e_end_time_eventi.append(dati[classe][csv]["Support"][evento])
                        audio_support_events=lista_start_e_end_time_eventi
    return audio_support_events

def get_query_set_inference(audio_name):
    audio_query_events=[]
    with open(val_json, 'r') as file:
        dati = json.load(file)
        for classe,value in dati.items():
            for csv in dati[classe]:
                if(dati[classe][csv]["Query"]!={}):
                    if dati[classe][csv]["Audio"] ==audio_name:

                        lista_start_e_end_time_eventi=[]
                        for evento in dati[classe][csv]["Query"]:
                            lista_start_e_end_time_eventi.append(dati[classe][csv]["Query"][evento])
                        audio_query_events=lista_start_e_end_time_eventi
    return audio_query_events         

### Audio Labeling

In [83]:
def label_audio(support_set,query_set,audio,frame_length=0.01):
    labels=[]
    class_dictionary={}
    class_dictionary['UNKNOWN']=0
    audio_path=os.path.join(val_dir,audio)
    audio_loaded, sr=librosa.load(audio_path)
    length_in_seconds=len(audio_loaded) / sr

    number_of_frames=math.ceil(length_in_seconds/frame_length)
    for i in range(number_of_frames):
        labels.append(0)
    label_classe=1
    
    for event in support_set:
        event_start=event['start']
        event_end=event['end']
        frame_start=int(event_start/frame_length)
        frame_end=int(event_end/frame_length)

        for i in range (frame_start,frame_end+1):
            labels[i]=label_classe



    for event in query_set:
        event_start=event['start']
        event_end=event['end']
        frame_start=int(event_start/frame_length)
        frame_end=int(event_end/frame_length)

        for i in range (frame_start,frame_end):
            labels[i]=label_classe
    return labels

## Support and query set creation in Spectrograms format. Uniform spectrogram creation.

In [84]:
def create_support_and_query_set_inference_audio_spectrograms(labels,audio_name, support_set_events,num_support_per_class=5,pcen=False,augment_negative_support=False):

    if pcen==False:
        if not os.path.exists(directory_path_spectrograms_inference):
            os.makedirs(directory_path_spectrograms_inference)
    else: 
         if not os.path.exists(directory_path_spectrograms_inference_pcen):
            os.makedirs(directory_path_spectrograms_inference_pcen)

   
    print(f" Inizio creazione support e query set spectrograms per l'audio: {audio_name}")

    path=os.path.join(val_dir,audio_name)
    audio_raw,sr = librosa.load(path)
    events=support_set_events[:num_support_per_class]
    end_support=0
    start_negative_support_set=0
    support_spectrograms=[]
    negative_support_spectrograms=[]


    mean_duration=0
    for evento in events:
        start=int(evento['start']*sr)
        end=int(evento['end']*sr)
        if mean_duration==0: mean_duration=end-start
        else: mean_duration+=end-start
    mean_duration=mean_duration/5

    for evento in events:
        start=int(evento['start']*sr)
        end=int(evento['end']*sr)
        
        #creation support set spectrograms for "unknown" class.
        audio_event=audio_raw[start_negative_support_set:start]
        
        """
        negative_frames=get_frames(audio_event,220,220)
        for frame in negative_frames:
        """

        if augment_negative_support==True:
            audios_augmented=augment_audio(audio_event,sr)
            for audio_augmented in audios_augmented:

                negative_audio_augmented=get_frames(audio_augmented,int(mean_duration),int(mean_duration))
                for negative_portion in negative_audio_augmented:
                    spectrogram_frames=librosa.feature.melspectrogram(y=negative_portion,n_fft=220,hop_length=110,n_mels=128)
                    if pcen==True:
                        spectrogram_frames = librosa.pcen(spectrogram_frames * (2**31))  
                    for  i in range(0,spectrogram_frames.shape[1],2):
                        end_frame = i + 2
                        if end_frame <= spectrogram_frames.shape[1]:
                            spectrogram_frame = spectrogram_frames[:, i:end_frame] 
                            spectrogram_frame=torch.tensor(spectrogram_frame)     
                            negative_support_spectrograms.append((spectrogram_frame,0))
                            spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                            spectrogram_augmented=torch.tensor(spectrogram_augmented)
                            negative_support_spectrograms.append((spectrogram_augmented,0))
        else: 
            negative_frames=get_frames(audio_event,int(mean_duration),int(mean_duration))
            for negative_frame in  negative_frames:
                spectrogram_frames=librosa.feature.melspectrogram(y=negative_frame,n_fft=220,hop_length=110,n_mels=128)
                if pcen==True:
                    spectrogram_frames = librosa.pcen(spectrogram_frames * (2**31))  #PCEN
        
                for i in range(0,spectrogram_frames.shape[1],2):
                    end_frame = i + 2
                    if end_frame <= spectrogram_frames.shape[1]:
                        spectrogram_frame = spectrogram_frames[:, i:end_frame]
                        spectrogram_frame=torch.tensor(spectrogram_frame)
                        negative_support_spectrograms.append((spectrogram_frame,0))
                


        start_negative_support_set=end+1
        if(end>end_support):
            end_support=end

        #audio_portion containing the event for the support set
        audio_event=audio_raw[start:end]

         #without overlapping, hop_length=frame_lenght
        spectrograms=[]
        
        spectrogram_frames=librosa.feature.melspectrogram(y=audio_event,n_fft=220,hop_length=110,n_mels=128)
        if pcen==True:
            spectrogram_frames = librosa.pcen(spectrogram_frames * (2**31))  
        for  i in range(0,spectrogram_frames.shape[1],2):

            end_frame = i + 2
            if end_frame <= spectrogram_frames.shape[1]:
                spectrogram_frame = spectrogram_frames[:, i:end_frame]
                spectrogram_frame=torch.tensor(spectrogram_frame)
                spectrograms.append((spectrogram_frame,1))
                
                spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                spectrograms.append((spectrogram_augmented,1))



        #data augmentation section for raw audios, the objective is increasing nearly 20x fold the support dimension
        audios_augmented=augment_audio(audio_event,sr)
        for audio_augmented in audios_augmented:
            spectrogram_frames=librosa.feature.melspectrogram(y=audio_augmented,n_fft=220,hop_length=110,n_mels=128)
            if pcen==True:
                spectrogram_frames = librosa.pcen(spectrogram_frames * (2**31))  
            for  i in range(0,spectrogram_frames.shape[1],2):
                end_frame = i + 2
                if end_frame <= spectrogram_frames.shape[1]:
                    spectrogram_frame = spectrogram_frames[:, i:end_frame] 
                    spectrogram_frame=torch.tensor(spectrogram_frame)     
                    spectrograms.append((spectrogram_frame,1))
                    spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                    spectrogram_augmented=torch.tensor(spectrogram_augmented)
                    spectrograms.append((spectrogram_augmented,1))



        support_spectrograms.extend(spectrograms)
    #all negative spectrogram if this code is commented, otherwise balanced support set.

    """
    negative_samples_to_keep=len(support_spectrograms)
    print(negative_samples_to_keep)
    print(len(negative_support_spectrograms))
    if(len(negative_support_spectrograms)>=negative_samples_to_keep):
        negative_support_spectrograms=random.sample(negative_support_spectrograms,negative_samples_to_keep)
    """

    
    support_spectrograms.extend((negative_support_spectrograms))

    query_spectrograms=[]
    audio_query=audio_raw[end_support+220:]

    j=len(labels)
    k=int((end_support/22050)/0.01)
    x=0
    frames=get_frames(audio_query,int(mean_duration),int(mean_duration))

    for frame in frames:
        spectrogram_frames=librosa.feature.melspectrogram(y=frame,n_fft=220,hop_length=110,n_mels=128) #labels must go from labels(k) to labels(k+)
        if pcen==True:
            spectrogram_frames = librosa.pcen(spectrogram_frames * (2**31))  
        for i in range(0,spectrogram_frames.shape[1],2):
            end_frame = i + 2
            if end_frame <= spectrogram_frames.shape[1]:
                spectrogram_frame = spectrogram_frames[:, i:end_frame] 
                spectrogram_frame=torch.tensor(spectrogram_frame)    
                x+=1
                if(x%2)==0:
                    query_spectrograms.append((spectrogram_frame,labels[k]))
                    if(k+1>j):break
                    k+=1
                else: 
                    if labels[k]!=labels[k+1]: 
                        query_spectrograms.append((spectrogram_frame,1)) #there is a frame 50% 0 and 50% 1, must be choosen if put 0 or 1.
                    else: 
                        query_spectrograms.append((spectrogram_frame,labels[k]))

   


    audio_support_and_query_set=(support_spectrograms,query_spectrograms)
    file_name=f'inference_audio{audio_name}.pth'

    if pcen==False:
        file_path=os.path.join(directory_path_spectrograms_inference, file_name)
    else: 
        file_path=os.path.join(directory_path_spectrograms_inference_pcen, file_name)
    torch.save(audio_support_and_query_set,file_path)
    print("Creazione dati conclusa")
    return audio_support_and_query_set                   




## Support and query set creation for inference audio based on sample rate 16KHz, therefore suitable for AST

In [85]:
def create_support_and_query_set_inference_audio_spectrogramsAST(labels,audio_name, support_set_events,num_support_per_class=5,pcen=True,augment_negative_support=False):
    pcen=True
  

    if not os.path.exists(directory_path_spectrograms_inference_AST):
        os.makedirs(directory_path_spectrograms_inference_AST)

   
    print(f" Inizio creazione support e query set spectrograms per l'audio: {audio_name}")

    path=os.path.join(val_dir,audio_name)
    audio_raw,sr = librosa.load(path,sr=16000)
    events=support_set_events[:num_support_per_class]
    end_support=0
    start_negative_support_set=0
    support_spectrograms=[]
    negative_support_spectrograms=[]

    
    mean_duration=0
    for evento in events:
        start=int(evento['start']*sr)
        end=int(evento['end']*sr)
        if mean_duration==0: mean_duration=end-start
        else: mean_duration+=end-start
    mean_duration=mean_duration/5
    
    for evento in events:
        start=int(evento['start']*sr)
        end=int(evento['end']*sr)
        
        #creation support set spectrograms for "unknown" class.
        audio_event=audio_raw[start_negative_support_set:start]
        
        """
        negative_frames=get_frames(audio_event,220,220)
        for frame in negative_frames:
        """

        if augment_negative_support==True:
            audios_augmented=augment_audio(audio_event,sr)
            for audio_augmented in audios_augmented:

                negative_audio_augmented=get_frames(audio_augmented,int(mean_duration),int(mean_duration))
                for negative_portion in negative_audio_augmented:
                    spectrogram_frames=librosa.feature.melspectrogram(y=negative_portion,n_fft=220,hop_length=110,n_mels=128)
                    if pcen==True:
                        spectrogram_frames = librosa.pcen(spectrogram_frames * (2**31))  
                    for  i in range(0,spectrogram_frames.shape[1],2):
                        end_frame = i + 2
                        if end_frame <= spectrogram_frames.shape[1]:
                            spectrogram_frame = spectrogram_frames[:, i:end_frame] 
                            spectrogram_frame=torch.tensor(spectrogram_frame)     
                            negative_support_spectrograms.append((spectrogram_frame,0))
                            spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                            spectrogram_augmented=torch.tensor(spectrogram_augmented)
                            negative_support_spectrograms.append((spectrogram_augmented,0))
        else: 
            negative_frames=get_frames(audio_event,int(mean_duration),int(mean_duration))
            for negative_frame in  negative_frames:
                spectrogram_frames=librosa.feature.melspectrogram(y=negative_frame,n_fft=220,hop_length=110,n_mels=128)
                if pcen==True:
                    spectrogram_frames = librosa.pcen(spectrogram_frames * (2**31))  #PCEN
        
                for i in range(0,spectrogram_frames.shape[1],2):
                    end_frame = i + 2
                    if end_frame <= spectrogram_frames.shape[1]:
                        spectrogram_frame = spectrogram_frames[:, i:end_frame]
                        spectrogram_frame=torch.tensor(spectrogram_frame)
                        negative_support_spectrograms.append((spectrogram_frame,0))
                


        start_negative_support_set=end+1
        if(end>end_support):
            end_support=end

        #audio_portion containing the event for the support set
        audio_event=audio_raw[start:end]

         #without overlapping, hop_length=frame_lenght
        spectrograms=[]
        
        spectrogram_frames=librosa.feature.melspectrogram(y=audio_event,n_fft=220,hop_length=110,n_mels=128)
        if pcen==True:
            spectrogram_frames = librosa.pcen(spectrogram_frames * (2**31))  
        for  i in range(0,spectrogram_frames.shape[1],2):

            end_frame = i + 2
            if end_frame <= spectrogram_frames.shape[1]:
                spectrogram_frame = spectrogram_frames[:, i:end_frame]
                spectrogram_frame=torch.tensor(spectrogram_frame)
                spectrograms.append((spectrogram_frame,1))
                
                spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                spectrograms.append((spectrogram_augmented,1))



        #data augmentation section for raw audios, the objective is increasing nearly 20x fold the support dimension
        audios_augmented=augment_audio(audio_event,sr)
        for audio_augmented in audios_augmented:
            spectrogram_frames=librosa.feature.melspectrogram(y=audio_augmented,n_fft=220,hop_length=110,n_mels=128)
            if pcen==True:
                spectrogram_frames = librosa.pcen(spectrogram_frames * (2**31))  
            for  i in range(0,spectrogram_frames.shape[1],2):
                end_frame = i + 2
                if end_frame <= spectrogram_frames.shape[1]:
                    spectrogram_frame = spectrogram_frames[:, i:end_frame] 
                    spectrogram_frame=torch.tensor(spectrogram_frame)     
                    spectrograms.append((spectrogram_frame,1))
                    spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                    spectrogram_augmented=torch.tensor(spectrogram_augmented)
                    spectrograms.append((spectrogram_augmented,1))



        support_spectrograms.extend(spectrograms)
    #all negative spectrogram if this code is commented, otherwise balanced support set.

    """
    negative_samples_to_keep=len(support_spectrograms)
    print(negative_samples_to_keep)
    print(len(negative_support_spectrograms))
    if(len(negative_support_spectrograms)>=negative_samples_to_keep):
        negative_support_spectrograms=random.sample(negative_support_spectrograms,negative_samples_to_keep)
    """

    
    support_spectrograms.extend((negative_support_spectrograms))

    query_spectrograms=[]
    audio_query=audio_raw[end_support+220:]

    j=len(labels)
    k=int((end_support/16000)/0.01)
    x=0
    frames=get_frames(audio_query,int(mean_duration),int(mean_duration))

    for frame in frames:
        spectrogram_frames=librosa.feature.melspectrogram(y=frame,n_fft=220,hop_length=110,n_mels=128) #labels must go from labels(k) to labels(k+)
        if pcen==True:
            spectrogram_frames = librosa.pcen(spectrogram_frames * (2**31))  
        for i in range(0,spectrogram_frames.shape[1],2):
            end_frame = i + 2
            if end_frame <= spectrogram_frames.shape[1]:
                spectrogram_frame = spectrogram_frames[:, i:end_frame] 
                spectrogram_frame=torch.tensor(spectrogram_frame)    
                x+=1
                if(x%2)==0:
                    query_spectrograms.append((spectrogram_frame,labels[k]))
                    if(k+1>j):break
                    k+=1
                else: 
                    if labels[k]!=labels[k+1]: 
                        query_spectrograms.append((spectrogram_frame,1)) #there is a frame 50% 0 and 50% 1, must be choosen if put 0 or 1.
                    else: 
                        query_spectrograms.append((spectrogram_frame,labels[k]))

   


    audio_support_and_query_set=(support_spectrograms,query_spectrograms)
    file_name=f'inference_audio{audio_name}.pth'

   
    file_path=os.path.join(directory_path_spectrograms_inference_AST, file_name)
    torch.save(audio_support_and_query_set,file_path)
    print("Creazione dati conclusa")
    return audio_support_and_query_set                   



## Divergent spectrogram creation.

In [86]:
def create_support_and_query_set_inference_audio_spectrograms_not_uniform(labels,audio_name, support_set_events,num_support_per_class=5):
            
    if not os.path.exists(directory_path_spectrograms_inference):
        os.makedirs(directory_path_spectrograms_inference)

   
    print(f" Inizio creazione support e query set spectrograms per l'audio: {audio_name}")

    path=os.path.join(val_dir,audio_name)
    audio_raw,sr = librosa.load(path)
    events=support_set_events[:num_support_per_class]
    end_support=0
    start_negative_support_set=0
    support_spectrograms=[]
    negative_support_spectrograms=[]


    mean_duration=0
    for evento in events:
        start=int(evento['start']*sr)
        end=int(evento['end']*sr)
        if mean_duration==0: mean_duration=end-start
        else: mean_duration+=end-start
    mean_duration=mean_duration/5

    for evento in events:
        start=int(evento['start']*sr)
        end=int(evento['end']*sr)
        
        #creation support set spectrograms for "unknown" class.
        audio_event=audio_raw[start_negative_support_set:start]
        
        """
        negative_frames=get_frames(audio_event,220,220)
        for frame in negative_frames:
        """
        negative_frames=get_frames(audio_event,sr*mean_duration,sr*mean_duration)

        for negative_frame in  negative_frames:
            spectrogram_frames=librosa.feature.melspectrogram(y=negative_frame,n_fft=220,hop_length=110,n_mels=128)
        #spectrogram_frames=torch.tensor(spectrogram_frames)
            for i in range(0,spectrogram_frames.shape[1],2):
                end_frame = i + 2
                if end_frame <= spectrogram_frames.shape[1]:
                    spectrogram_frame = spectrogram_frames[:, i:end_frame]
                    negative_support_spectrograms.append((spectrogram_frame,0))
            


        start_negative_support_set=end+1
        if(end>end_support):
            end_support=end

        #audio_portion containing the event for the support set
        audio_event=audio_raw[start:end]

         #without overlapping, hop_length=frame_lenght
        spectrograms=[]
        
        spectrogram_frames=librosa.feature.melspectrogram(y=audio_event,n_fft=220,hop_length=110,n_mels=128)
        for  i in range(0,spectrogram_frames.shape[1],2):

            end_frame = i + 2
            if end_frame <= spectrogram_frames.shape[1]:
                spectrogram_frame = spectrogram_frames[:, i:end_frame]
                
                spectrograms.append((spectrogram_frame,1))
                spectrogram_frame=torch.tensor(spectrogram_frame)
                spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                spectrograms.append((spectrogram_augmented,1))



        #data augmentation section for raw audios, the objective is increasing nearly 20x fold the support dimension
        audios_augmented=augment_audio(audio_event,sr)
        for audio_augmented in audios_augmented:
            
           
            spectrogram_frames=librosa.feature.melspectrogram(y=audio_augmented,n_fft=220,hop_length=110,n_mels=128)
            for  i in range(0,spectrogram_frames.shape[1],2):

                end_frame = i + 2
                if end_frame <= spectrogram_frames.shape[1]:
                    spectrogram_frame = spectrogram_frames[:, i:end_frame]      
                    spectrograms.append((spectrogram_frame,1))
                    spectrogram_augmented=augment_spectrogram(spectrogram_frame)
                    spectrograms.append((spectrogram_augmented,1))



        support_spectrograms.extend(spectrograms)
    """
    negative_samples_to_keep=len(support_spectrograms)
    print(negative_samples_to_keep)
    print(len(negative_support_spectrograms))
    if(len(negative_support_spectrograms)>=negative_samples_to_keep):
        negative_support_spectrograms=random.sample(negative_support_spectrograms,negative_samples_to_keep)
    """



    
    negative_samples_to_keep=len(support_spectrograms)
    print(negative_samples_to_keep)
    print(len(negative_support_spectrograms))
    if(len(negative_support_spectrograms)>=negative_samples_to_keep):
        negative_support_spectrograms=random.sample(negative_support_spectrograms,negative_samples_to_keep)



    support_spectrograms.extend((negative_support_spectrograms))

    audio_query=audio_raw[end_support+220:]

    frames=get_frames(audio_query,220,220)
    query_spectrograms=[]
    k=0
    print(len(frames))
    print(len(labels)-(end_support/22050)/0.01)

    for frame_id in range(int((end_support/22050)/0.01),len(labels)):
        spectrogram_frame=librosa.feature.melspectrogram(y=frames[k],n_fft=220,hop_length=220,n_mels=128)
        query_spectrograms.append((spectrogram_frame,labels[frame_id]))
        k=k+1

    audio_support_and_query_set=(support_spectrograms,query_spectrograms)
    file_name=f'inference_audio{audio_name}.pth'
    file_path=os.path.join(directory_path_spectrograms_inference, file_name)
    torch.save(audio_support_and_query_set,file_path)
    print("Creazione dati conclusa")
    return audio_support_and_query_set     

## Support and query set creation in Raw Audio format

In [87]:
def create_support_and_query_set_inference_audio_raw(labels,audio_name, support_set_events,num_support_per_class=5):

    if not os.path.exists(directory_path_raw_audio_inference):
        os.makedirs(directory_path_raw_audio_inference)
        

    print(f" Inizio creazione support e query set spectrograms per l'audio: {audio_name}")

    path=os.path.join(val_dir,audio_name)
    audio_raw,sr = librosa.load(path,sr=16000)
    events=support_set_events[:num_support_per_class]
    end_support=0
    start_negative_support_set=0
    support_raw_audios=[]
    negative_support_raw_audios=[]
    for evento in events:
        start=int(evento['start']*sr)
        end=int(evento['end']*sr)

        #creation support set spectrograms for "unknown" class.
        audio_event=audio_raw[start_negative_support_set:start]
        negative_frames=get_frames(audio_event,160,160)
        for frame in negative_frames:
           negative_support_raw_audios.append((frame,0))



        start_negative_support_set=end+1
        if(end>end_support):
            end_support=end

        #audio_portion containing the event for the support set
        audio_event=audio_raw[start:end]

        frames=get_frames(audio_event,160,160) #without overlapping, hop_length=frame_lenght
        raw_audios=[]
        for frame in frames:
            raw_audios.append((frame,1))



        #data augmentation section for raw audios, the objective is increasing nearly 20x fold the support dimension
        audios_augmented=augment_audio(audio_event,sr)
        for audio_augmented in audios_augmented:
            frames=get_frames(audio_augmented,160,160)
            for frame in frames:
                raw_audios.append((frame,1))



        support_raw_audios.extend(raw_audios)

    

    support_raw_audios.extend((negative_support_raw_audios))

    audio_query=audio_raw[end_support+160:]

    frames=get_frames(audio_query,160,160)
    raw_audio_query=[]
    k=0
   

    for frame_id in range(int((end_support/16000)/0.01),len(frames)+int((end_support/16000)/0.01)):
        frame=frames[k]
        raw_audio_query.append((frame,labels[frame_id]))
        k=k+1

    audio_support_and_query_set=(support_raw_audios,raw_audio_query)
    file_name=f'inference_audio{audio_name}.pth'
    file_path=os.path.join(directory_path_raw_audio_inference, file_name)
    torch.save(audio_support_and_query_set,file_path)
    return audio_support_and_query_set   

## Metric section

In [88]:
import numpy as np
import mir_eval
import scipy


def fast_intersect(ref, est):
    """Find all intersections between reference events and estimated events (fast).
    Best-case complexity: O(N log N + M log M) where N=length(ref) and M=length(est)

    Parameters
    ----------
    ref: np.ndarray [shape=(2, n)], real-valued
         Array of reference events. Each column is an event.
         The first row denotes onset times and the second row denotes offset times.

    est: np.ndarray [shape=(2, m)], real-valued
         Array of estimated events. Each column is an event.
         The first row denotes onset times and the second row denotes offset times.


    Returns
    -------
    matches: list of sets, length n, integer-valued
         Property: matches[i] contains the set of all indices j such that
            (ref[0, i]<=est[1, j]) AND (ref[1, i]>=est[0, j])
    """
    ref_on_argsort = np.argsort(ref[0, :])
    ref_off_argsort = np.argsort(ref[1, :])

    est_on_argsort = np.argsort(est[0, :])
    est_off_argsort = np.argsort(est[1, :])

    est_on_maxindex = est.shape[1]
    est_off_minindex = 0
    estref_matches = [set()] * ref.shape[1]
    refest_matches = [set()] * ref.shape[1]
    for ref_id in range(ref.shape[1]):
        ref_onset = ref[0, ref_on_argsort[ref_id]]
        est_off_sorted = est[1, est_off_argsort[est_off_minindex:]]
        search_result = np.searchsorted(est_off_sorted, ref_onset, side="left")
        est_off_minindex += search_result
        refest_match = est_off_argsort[est_off_minindex:]
        refest_matches[ref_on_argsort[ref_id]] = set(refest_match)

        ref_offset = ref[1, ref_off_argsort[-1 - ref_id]]
        est_on_sorted = est[0, est_on_argsort[: (1 + est_on_maxindex)]]
        search_result = np.searchsorted(est_on_sorted, ref_offset, side="right")
        est_on_maxindex = search_result - 1
        estref_match = est_on_argsort[: (1 + est_on_maxindex)]
        estref_matches[ref_off_argsort[-1 - ref_id]] = set(estref_match)

    zip_iterator = zip(refest_matches, estref_matches)
    matches = [x.intersection(y) for (x, y) in zip_iterator]
    return matches


def iou(ref, est, method="fast"):
    """Compute pairwise "intersection over union" (IOU) metric between reference
    events and estimated events.

    Let us denote by a_i and b_i the onset and offset of reference event i.
    Let us denote by u_j and v_j the onset and offset of estimated event j.

    The IOU between events i and j is defined as
        (min(b_i, v_j)-max(a_i, u_j)) / (max(b_i, v_j)-min(a_i, u_j))
    if the events are non-disjoint, and equal to zero otherwise.

    Parameters
    ----------
    ref: np.ndarray [shape=(2, n)], real-valued
         Array of reference events. Each column is an event.
         The first row denotes onset times and the second row denotes offset times.

    est: np.ndarray [shape=(2, m)], real-valued
         Array of estimated events. Each column is an event.
         The first row denotes onset times and the second row denotes offset times.

    method: str, optional.
         If "fast" (default), computes pairwise intersections via a custom
         dynamic programming algorithm, see fast_intersect.
         If "slow", computes pairwise intersections via bruteforce quadratic
         search, see slow_intersect.

    Returns
    -------
    S: scipy.sparse.dok.dok_matrix, real-valued
        Sparse 2-D matrix. S[i,j] contains the IOU between ref[i] and est[j]
        if these events are non-disjoint and zero otherwise.
    """
    n_refs = ref.shape[1]
    n_ests = est.shape[1]
    S = scipy.sparse.dok_matrix((n_refs, n_ests))

    if method == "fast":
        matches = fast_intersect(ref, est)
    elif method == "slow":
        matches = slow_intersect(ref, est)

    for ref_id in range(n_refs):
        matching_ests = matches[ref_id]
        ref_on = ref[0, ref_id]
        ref_off = ref[1, ref_id]

        for matching_est_id in matching_ests:
            est_on = est[0, matching_est_id]
            est_off = est[1, matching_est_id]
            intersection = min(ref_off, est_off) - max(ref_on, est_on)
            union = max(ref_off, est_off) - min(ref_on, est_on)
            intersection_over_union = intersection / union
            S[ref_id, matching_est_id] = intersection_over_union

    return S


def match_events(ref, est, min_iou=0.0, method="fast"):
    """
    Compute a maximum matching between reference and estimated event times,
    subject to a criterion of minimum intersection-over-union (IOU).

    Given two lists of events ``ref`` (reference) and ``est`` (estimated),
    we seek the largest set of correspondences ``(ref[i], est[j])`` such that
        ``iou(ref[i], est[j]) <= min_iou``
    and such that each ``ref[i]`` and ``est[j]`` is matched at most once.

    This function is strongly inspired by mir_eval.onset.util.match_events.
    It relies on mir_eval's implementation of the Hopcroft-Karp algorithm from
    maximum bipartite graph matching. However, one important difference is that
    mir_eval's distance function relies purely on onset times, whereas this function
    considers both onset times and offset times to compute the IOU metric between
    reference events and estimated events.

    Parameters
    ----------
    ref: np.ndarray [shape=(2, n)], real-valued
         Array of reference events. Each column is an event.
         The first row denotes onset times and the second row denotes offset times.

    est: np.ndarray [shape=(2, m)], real-valued
         Array of estimated events. Each column is an event.
         The first row denotes onset times and the second row denotes offset times.

    min_iou: real number in [0, 1). Default: 0.
         Threshold for minimum amount of intersection over union (IOU) to match
         any two events. See the iou method for implementation details.

    method: str, optional.
         If "fast" (default), computes pairwise intersections via a custom
         dynamic programming algorithm, see fast_intersect.
         If "slow", computes pairwise intersections via bruteforce quadratic
         search, see slow_intersect.

    Returns
    -------
    matching : list of tuples
        Every tuple corresponds to a match between one reference event and
        one estimated event.
            ``matching[i] == (i, j)`` where ``ref[i]`` matches ``est[j]``.
        Note that all values i and j appear at most once in the list.
    """

    # Intersect reference events and estimated events
    S = iou(ref, est, method=method)

    # Threshold intersection-over-union (IOU) ratio
    S_bool = scipy.sparse.dok_matrix(S > min_iou)
    hits = S_bool.keys()

    # Construct the bipartite graph
    G = {}
    for ref_i, est_i in hits:
        if est_i not in G:
            G[est_i] = []
        G[est_i].append(ref_i)

    # Apply Hopcroft-Karp algorithm (from mir_eval package)
    # to obtain maximum bipartite graph matching
    matching = sorted(mir_eval.util._bipartite_match(G).items())
    return matching


def slow_intersect(ref, est):
    """Find all intersections between reference events and estimated events (slow).
    Best-case complexity: O(N*M) where N=ref.shape[1] and M=est.shape[1]

    Parameters
    ----------
    ref: np.ndarray [shape=(2, n)], real-valued
         Array of reference events. Each column is an event.
         The first row denotes onset times and the second row denotes offset times.

    est: np.ndarray [shape=(2, m)], real-valued
         Array of estimated events. Each column is an event.
         The first row denotes onset times and the second row denotes offset times.


    Returns
    -------
    matches: list of sets, length n, integer-valued
         Property: matches[i] contains the set of all indices j such that
            (ref[0, i]<=est[1, j]) AND (ref[1, i]>=est[0, j])
    """
    matches = []
    for i in range(ref.shape[1]):
        matches.append(
            set(
                [
                    j
                    for j in range(est.shape[1])
                    if ((ref[0, i] <= est[1, j]) and (ref[1, i] >= est[0, j]))
                ]
            )
        )
    return matches

In [89]:


MIN_EVAL_VALUE = 0.00001
N_SHOTS = 5
MIN_IOU_TH = 0.3
PRED_FILE_HEADER = ["Audiofilename","Starttime","Endtime"]
POS_VALUE = 'POS'
UNK_VALUE = 'UNK'

def remove_shots_from_ref(ref_df, number_shots=5):
    
    ref_pos_indexes = select_events_with_value(ref_df, value=POS_VALUE)
    ref_n_shot_index = ref_pos_indexes[number_shots-1]
    # remove all events (pos and UNK) that happen before this 5th event
    events_to_drop = ref_df.index[ref_df['Endtime'] <= ref_df.iloc[ref_n_shot_index]['Endtime']].tolist()

    return ref_df.drop(events_to_drop)

def select_events_with_value(data_frame, value=POS_VALUE):

    indexes_list = data_frame.index[data_frame["Q"] == value].tolist()

    return indexes_list

def build_matrix_from_selected_rows(data_frame, selected_indexes_list ):

    matrix_data = np.ones((2, len(selected_indexes_list)))* -1
    for n, idx in enumerate(selected_indexes_list):
        matrix_data[0, n] = data_frame.loc[idx].Starttime # start time for event n
        matrix_data[1, n] = data_frame.loc[idx].Endtime
    return matrix_data


def compute_tp_fp_fn(pred_events_df, ref_events_df):
    # inputs: dataframe with predicted events, dataframe with reference events and their value (POS, UNK, NEG)
    # output: True positives, False Positives, False negatives counts and total number of pos events in ref.

    # makes one pass with bipartite graph matching between pred events and ref positive events
    # get TP
    # make second pass with remaining pred events and ref Unk events
    # compute FP as the number of remaining predicted events after the two rounds of matches.
    # FN is the remaining unmatched pos events in ref.

    ref_pos_indexes = select_events_with_value(ref_events_df, value=POS_VALUE)

    if "Q" not in pred_events_df.columns:
        pred_events_df["Q"] = POS_VALUE

    #sort events by starttime
    pred_events_df = pred_events_df.sort_values(by='Starttime', axis=0, ascending=True)
    pred_pos_indexes = select_events_with_value(pred_events_df, value=POS_VALUE)

    ref_1st_round = build_matrix_from_selected_rows(ref_events_df, ref_pos_indexes)
    pred_1st_round = build_matrix_from_selected_rows(pred_events_df, pred_pos_indexes)

    m_pos = metrics.match_events(ref_1st_round, pred_1st_round, min_iou=MIN_IOU_TH)
    matched_ref_indexes = [ri for ri, pi in m_pos] 
    matched_pred_indexes = [pi for ri, pi in m_pos]


    ref_unk_indexes = select_events_with_value(ref_events_df, value=UNK_VALUE)
    ref_2nd_round = build_matrix_from_selected_rows(ref_events_df, ref_unk_indexes)

    unmatched_pred_events = list(set(range(pred_1st_round.shape[1])) - set(matched_pred_indexes))
    pred_2nd_round = pred_1st_round[:, unmatched_pred_events]

    m_unk = metrics.match_events(ref_2nd_round, pred_2nd_round, min_iou=MIN_IOU_TH)

    # print("# Positive matches between Ref and Pred :", len(m_pos))
    # print("# matches with Unknown events: ", len(m_unk))
    
    tp = len(m_pos)
    fp = pred_1st_round.shape[1] - tp - len(m_unk)
    
    ## compute unmatched pos ref events:
    count_unmached_pos_ref_events = len(ref_pos_indexes) - tp

    fn = count_unmached_pos_ref_events

    total_n_POS_events = len(ref_pos_indexes)
    return tp, fp, fn, total_n_POS_events

def compute_scores_per_class(counts_per_class):

    scores_per_class = {}
    for cl in counts_per_class.keys():
        tp = counts_per_class[cl]["TP"]
        fp = counts_per_class[cl]["FP"]
        fn = counts_per_class[cl]["FN"]

            
        # to compute the harmonic mean we need to have all entries as non zero
        precision = tp/(tp+fp) if tp+fp != 0 else MIN_EVAL_VALUE  # case where no predictions were made 
        if precision < MIN_EVAL_VALUE:
            precision = MIN_EVAL_VALUE
        recall = tp/(fn+tp) if tp != 0 else MIN_EVAL_VALUE
        fmeasure = tp/(tp+0.5*(fp+fn)) if tp != 0 else MIN_EVAL_VALUE
        
        scores_per_class[cl] = {"precision": precision, "recall": recall, "f-measure": fmeasure}

    return scores_per_class
    
def compute_scores_from_counts(counts):
    tp = counts["TP"]
    fp = counts["FP"]
    fn = counts["FN"]

    # to compute the harmonic mean we need to have all entries as non zero
    precision = tp/(tp+fp) if tp+fp != 0 else MIN_EVAL_VALUE  # case where no predictions were made 
    if precision < MIN_EVAL_VALUE:
        precision = MIN_EVAL_VALUE 
    recall = tp/(fn+tp) if tp != 0 else MIN_EVAL_VALUE
    fmeasure = tp/(tp+0.5*(fp+fn)) if tp != 0 else MIN_EVAL_VALUE

    scores = {"precision": precision, "recall": recall, "f-measure": fmeasure}
    
    return scores


def build_report(main_set_scores, scores_per_miniset, scores_per_audiofile, save_path, main_set_name="EVAL", team_name="test_team" , **kwargs):
    

    # datetime object containing current date and time
    now = datetime.now()
    date_string = now.strftime("%d%m%Y_%H_%M_%S")
    # print("date and time =", date_string)	

    #make dict:
    report = {
            'team_name': team_name,
            "set_name": main_set_name,
            "report_date": date_string,
            "overall_scores": main_set_scores,
            "scores_per_subset": scores_per_miniset,
            "scores_per_audiofile": scores_per_audiofile
    }
    if "scores_per_class" in kwargs.keys():
        report["scores_per_class"] = kwargs['scores_per_class']

    with open(os.path.join(save_path,"Evaluation_report_" + team_name + "_" + main_set_name + '_' + date_string + '.json'), 'w') as outfile:
        json.dump(report, outfile)

    return

def build_mini_report_bootstrapped_results(low, high, meanFmeasure, low_precision,  mean_precision, high_precision, low_recall, 
                                    mean_recall, high_recall, save_path, main_set_name="EVAL", team_name="test_team" ):
    

    # datetime object containing current date and time
    now = datetime.now()
    date_string = now.strftime("%d%m%Y_%H_%M_%S")
    # print("date and time =", date_string)	

    #make dict:
    report = {
            'team_name': team_name,
            "set_name": main_set_name,
            "report_date": date_string,
            "fmeasure": {"low": low, "mean": meanFmeasure, "high":high},
            "precision":  {"low": low_precision, "mean": mean_precision, "high":high_precision},
            "recall":  {"low": low_recall, "mean": mean_recall, "high":high_recall}
    }
    
    with open(os.path.join(save_path,"Evaluation_report_" + team_name + "_" + main_set_name + '_' + date_string + '.json'), 'w') as outfile:
        json.dump(report, outfile)

    return

def evaluate_bootstrapped(pred_file_path, ref_file_path, team_name, dataset, savepath, bootstraps = 1000):

    #computes overall scores with 95% confidence intervals
    #generates report.

    print("\nEvaluation for:", team_name, dataset)
    #read Gt file structure: get subsets and paths for ref csvs make an inverted dictionary with audiofilenames as keys and folder as value
    gt_file_structure = {}
    gt_file_structure[dataset] = {}
    inv_gt_file_structure = {}
    list_of_subsets = os.listdir(ref_file_path)
    for subset in list_of_subsets:
        gt_file_structure[dataset][subset] = [os.path.basename(fl)[0:-4]+'.wav' for fl in glob.glob(os.path.join(ref_file_path,subset,"*.csv"))]
        for audiofile in gt_file_structure[dataset][subset]:
            inv_gt_file_structure[audiofile] = subset


    #read prediction csv
    pred_csv = pd.read_csv(pred_file_path, dtype=str)
    #verify headers:
    if list(pred_csv.columns) !=  PRED_FILE_HEADER:
        print('Please correct the header of the prediction file. This should be', PRED_FILE_HEADER)
        exit(1)
    
    overall_fmeasures_bootstrapped = []
    overall_precision_bootstrapped = []
    overall_recall_bootstrapped = []

    for bi in tqdm(range(bootstraps)):
        #  parse prediction csv

        #remove predictions for bootstrapping 
        prediction_indexes_to_remove = np.random.choice(range(len(pred_csv)), round(0.05*len(pred_csv)), replace=False)

        pred_csv_new = pred_csv.drop(prediction_indexes_to_remove)
        #  split remaining predictions into lists of events for the same audiofile.
        pred_events_by_audiofile = dict(tuple(pred_csv_new.groupby('Audiofilename')))

        counts_per_audiofile = {}
        for audiofilename in list(pred_events_by_audiofile.keys()):
        
                
            # for each audiofile, load correcponding GT File (audiofilename.csv)
            ref_events_this_audiofile_all = pd.read_csv(os.path.join(ref_file_path, inv_gt_file_structure[audiofilename], audiofilename[0:-4]+'.csv'), dtype={'Starttime':np.float64, 'Endtime': np.float64})
            # sort events by starttime:
            ref_events_this_audiofile_all = ref_events_this_audiofile_all.sort_values(by='Starttime', axis=0, ascending=True)
            
            #Remove the 5 shots from GT:
            ref_events_this_audiofile = remove_shots_from_ref(ref_events_this_audiofile_all, number_shots=N_SHOTS)
            
            # compare and get counts: TP, FP .. 
            tp_count, fp_count, fn_count , total_n_events_in_audiofile = compute_tp_fp_fn(pred_events_by_audiofile[audiofilename], ref_events_this_audiofile )

            counts_per_audiofile[audiofilename] = {"TP": tp_count, "FP": fp_count, "FN": fn_count, "total_n_pos_events": total_n_events_in_audiofile}
            # print(audiofilename, counts_per_audiofile[audiofilename])

        dataset_metadata = copy.deepcopy(gt_file_structure)

        # include audiofiles for which there were no predictions:
        list_all_audiofiles = []
        for miniset in dataset_metadata[dataset].keys():
            list_all_audiofiles.extend(dataset_metadata[dataset][miniset])

        for audiofilename in list_all_audiofiles:
            if audiofilename not in counts_per_audiofile.keys():
                ref_events_this_audiofile = pd.read_csv(os.path.join(ref_file_path, inv_gt_file_structure[audiofilename], audiofilename[0:-4]+'.csv'), dtype=str)
                # sort ref_events by starttime
                ref_events_this_audiofile = ref_events_this_audiofile.sort_values(by='Starttime', axis=0, ascending=True)
                total_n_pos_events_in_audiofile =  len(select_events_with_value(ref_events_this_audiofile, value=POS_VALUE))
                counts_per_audiofile[audiofilename] = {"TP": 0, "FP": 0, "FN": total_n_pos_events_in_audiofile, "total_n_pos_events": total_n_pos_events_in_audiofile}
            
        # aggregate the counts per class or subset: 
        list_sets_in_mainset = list(dataset_metadata[dataset].keys())

        counts_per_class_per_set = {}
        scores_per_class_per_set = {}
        counts_per_set = {}
        scores_per_set = {}
        scores_per_audiofile = {}
        for data_set in list_sets_in_mainset:
            # print(data_set)
            
            list_audiofiles_in_set = dataset_metadata[dataset][data_set]
            tp = 0
            fn = 0
            fp = 0
            total_n_pos_events_this_set = 0
            for audiofile in  list_audiofiles_in_set:

                scores_per_audiofile[audiofile] = compute_scores_from_counts(counts_per_audiofile[audiofile])
                tp = tp + counts_per_audiofile[audiofile]["TP"]
                fn = fn + counts_per_audiofile[audiofile]["FN"]
                fp = fp + counts_per_audiofile[audiofile]["FP"]
                total_n_pos_events_this_set = total_n_pos_events_this_set + counts_per_audiofile[audiofile]["total_n_pos_events"]
                counts_per_set[data_set] = {"TP": tp, "FN": fn, "FP": fp, "total_n_pos_events_this_set": total_n_pos_events_this_set}
            
                #  compute scores per subset
                scores_per_set[data_set] = compute_scores_from_counts(counts_per_set[data_set])
                        
        overall_scores = {"precision" : stats.hmean([scores_per_set[dt]["precision"] for dt in scores_per_set.keys()]), 
                        "recall":  stats.hmean([scores_per_set[dt]["recall"] for dt in scores_per_set.keys()]) ,
                        "fmeasure (percentage)": np.round(stats.hmean([scores_per_set[dt]["f-measure"] for dt in scores_per_set.keys()])*100, 3)
                        }
        overall_fmeasures_bootstrapped.append(overall_scores["fmeasure (percentage)"])
        overall_precision_bootstrapped.append(overall_scores["precision"])
        overall_recall_bootstrapped.append(overall_scores["recall"])
    
    overall_high_fmeasure = np.percentile(overall_fmeasures_bootstrapped,97.5)
    overall_low_fmeasure = np.percentile(overall_fmeasures_bootstrapped, 2.5)
    overall_mean_fmeasure = np.mean(overall_fmeasures_bootstrapped)
    overall_high_precision = np.percentile(overall_precision_bootstrapped,97.5)
    overall_low_precision = np.percentile(overall_precision_bootstrapped, 2.5)
    overall_mean_precision = np.mean(overall_precision_bootstrapped)
    overall_high_recall = np.percentile(overall_recall_bootstrapped,97.5)
    overall_low_recall = np.percentile(overall_recall_bootstrapped, 2.5)
    overall_mean_recall = np.mean(overall_recall_bootstrapped)

    # print(overall_low_fmeasure, '<',overall_mean_fmeasure, '<', overall_high_fmeasure )
    # print('min:', np.min(overall_fmeasures_bootstrapped), '------ max:', np.max(overall_fmeasures_bootstrapped))
    build_mini_report_bootstrapped_results(overall_low_fmeasure, overall_high_fmeasure, overall_mean_fmeasure,
                                     overall_high_precision, overall_low_precision,  overall_mean_precision, overall_high_recall,
                                     overall_low_recall, overall_mean_recall, savepath, dataset, team_name)
       
    return


if __name__ == "__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument('-pred_file', type=str, help='csv predictions file')
    parser.add_argument('-ref_files_path', type=str, help='path to the ground truth csvs folder')
    # parser.add_argument('-metadata', type=str, help="path for metadata json. Participants may ignore this option.")
    parser.add_argument('-team_name', type=str, help='team identification')
    parser.add_argument('-dataset', type=str, help="which set to evaluate: EVAL or VAL")
    parser.add_argument('-savepath', type=str, help="path where to save the report to")
    args = parser.parse_args()
    
    

    evaluate_bootstrapped(args.pred_file_path,args.ref_file_path, args.team_name, args.dataset,  args.savepath)

usage: ipykernel_launcher.py [-h] [-pred_file PRED_FILE]
                             [-ref_files_path REF_FILES_PATH]
                             [-team_name TEAM_NAME] [-dataset DATASET]
                             [-savepath SAVEPATH]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\aldob\AppData\Roaming\jupyter\runtime\kernel-v3765acdaeec305910eb5ab3da3e48fe2de235787c.json


SystemExit: 2

C:\Users\aldob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Utility function to improve localization

In [123]:
def wrong_prediction_correction(predictions,mean_event_duration,mode=0):
    frame_mean_event_duration=int(mean_event_duration/0.01)
    if frame_mean_event_duration<4 or mode=="Wav2Vec" or mode=="AST":
        return predictions

    high_percentage_error_percentage = math.ceil(frame_mean_event_duration / 5)
    high_percentage_error_percentage=4
   
    if frame_mean_event_duration<70:high_percentage_error_percentage=2


    for p in range(len(predictions)):
        if(p<high_percentage_error_percentage or p+high_percentage_error_percentage>len(predictions)):
            continue
        
        correction_needed= False
        for q in range (-high_percentage_error_percentage,high_percentage_error_percentage):
            if p + q < 0 or p + q >= len(predictions):
                continue
            if predictions[p] != predictions[p + q]:
                correction_needed = True
                break

            
        if correction_needed==True:
            predictions[p]=1 -predictions[p]
            
    return predictions



## Event handling function

In [124]:
def event_finder(label_vector,mean_event,mode=0):
    print(int((mean_event)/0.01))
    y=int(mean_event/0.01)

    
    if 1500<int(mean_event/0.01)<2000:min_event_length_frames=55
    elif int(mean_event/0.01)>2000:min_event_length_frames=65
    elif 1000<int(mean_event/0.01)<1500:min_event_length_frames=45
    elif int((mean_event)/0.01)<40:min_event_length_frames=int((mean_event)/0.01)*0.75
    elif int((mean_event)/0.01)<30:min_event_length_frames=25
    
    if mode=="Wav2Vec":
        if int(mean_event/0.01)>2000:min_event_length_frames=30
        #elif 1600<y<2000: min_event_length_frames=20  #20=0.333
        elif 1000<y<2000:min_event_length_frames=50
        else: min_event_length_frames=y*0.75
    if mode=="AST":
        min_event_length_frames=20
    
    print(f'min_event_frame:{min_event_length_frames}')
    events = []
    start = None

    for number, label in enumerate(label_vector):
        if label == 1 and start is None:
            # Inizia un nuovo evento
            start = number
        elif label == 0 and start is not None:
            # Finisce l'evento corrente
            if number - start >= min_event_length_frames:
                events.append((start*0.01, (number-1)*0.01))
            start = None
            
    if start is not None and len(label_vector) - start >= min_event_length_frames:
        events.append((start * 0.01, (len(label_vector) - 1) * 0.01))


    return events


## Segmentation metrics

In [92]:
def iou(event1, event2):
    """Calcola l'Intersection over Union (IoU) tra due eventi (start, end)."""
    start1, end1 = event1
    start2, end2 = event2

    inter_start = max(start1, start2)
    inter_end = min(end1, end2)
    inter = max(0, inter_end - inter_start + 1)

    union = max(end1, end2) - min(start1, start2) + 1

    return inter / union

In [93]:
def compute_tp_fp_fn(pred_events, true_events, min_iou_th=0.05):
    """
    Calcola TP, FP, FN basato su IoU tra gli eventi predetti e quelli reali.
    
    Args:
    - pred_events (list of tuples): Lista di eventi predetti (start, end).
    - true_events (list of tuples): Lista di eventi reali (start, end).
    - min_iou_th (float): Soglia minima di IoU per considerare una predizione come vera positiva.
    
    Returns:
    - tp, fp, fn (int, int, int): True Positives, False Positives, False Negatives.
    """
    tp = 0
    matched_pred_indexes = set()

    for true_event in true_events:
        match_found = False
        for id, pred_event in enumerate(pred_events):
            if id in matched_pred_indexes:
                continue

            # Calcolo dell'IoU tra evento reale e evento predetto
            if iou(true_event, pred_event) >= min_iou_th:
                tp += 1
                matched_pred_indexes.add(id)
                match_found = True
                break
    
    fp = len(pred_events) - tp
    fn = len(true_events) - tp
    print(tp,fp,fn)
    return tp, fp, fn

In [94]:
def compute_precision_recall_f1(tp, fp, fn, min_eval_value=0.00001):
    """
    Calcola precision, recall, e F1-score basato su TP, FP, FN.
    
    Args:
    - tp (int): Numero di True Positives.
    - fp (int): Numero di False Positives.
    - fn (int): Numero di False Negatives.
    - min_eval_value (float): Valore minimo per evitare divisioni per 0.
    
    Returns:
    - metrics (dict): Dizionario con precision, recall e F1-score.
    """
    precision = tp / (tp + fp) if tp + fp != 0 else min_eval_value
    recall = tp / (tp + fn) if tp + fn != 0 else min_eval_value
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else min_eval_value

    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score
    }

## Inference function

In [104]:
def inference_audio(model,audio_name,mode="classical_network",model_checkpoint=0,batch_size=32,pcen=False,rolling_mean=False):
    model.eval()

    pth_file=f'inference_audio{audio_name}.pth'
    print(i)

    if mode=="AST": batch_size=512
    if mode=="Wav2Vec": batch_size=32

    if(model_checkpoint!=0):
        model.load_state_dict(model_checkpoint['model_state_dict'])
        optimizer.load_state_dict(model_checkpoint['optimizer_state_dict'])


    if( mode=="Wav2Vec"):
        file_path=os.path.join(directory_path_raw_audio_inference, pth_file)
    elif mode=="AST":
        file_path=os.path.join(directory_path_spectrograms_inference_AST, pth_file)
    else:
        if pcen==False:
            file_path=os.path.join(directory_path_spectrograms_inference, pth_file)
        else:
            file_path=os.path.join(directory_path_spectrograms_inference_pcen, pth_file)


    inference_audio=torch.load(file_path)
    query_predictions=[]


    support_samples = [x[0] for x in inference_audio[0]]
    support_labels = [x[1] for x in inference_audio[0]]

    query_samples = [x[0] for x in inference_audio[1]]
    query_labels =[x[1] for x in inference_audio[1]]

    
    if mode=="AST" or mode=="Wav2Vec":
        if mode=="AST":
            support_samples=torch.stack(support_samples).to(dtype=torch.float32).to(device)
            query_samples=torch.stack(query_samples).to(dtype=torch.float32).to(device)
            support_labels=torch.tensor(support_labels)
            print(support_samples.shape,support_labels.shape)
            mask_1 = support_labels == 1
            mask_0 = support_labels == 0    

        
        
            support_samples = torch.cat((support_samples[mask_1], support_samples[mask_0]), dim=0)
            support_labels = torch.cat((support_labels[mask_1], support_labels[mask_0]), dim=0)
        elif mode=="classical_network":
           
            support_samples=torch.stack(support_samples)
            support_labels=torch.tensor(support_labels)
        else: 
            support_samples = torch.tensor(support_samples,dtype=torch.float16).to(device)
            query_samples = torch.tensor(query_samples,dtype=torch.float16).to(device)
       


    
    del inference_audio

    if(mode=="classical_network"):
        
        support_samples=torch.stack(support_samples).to(dtype=torch.float32).to(device)
        query_samples=torch.stack(query_samples).to(dtype=torch.float32).to(device)
        
        #support_samples = torch.tensor(support_samples,dtype=torch.float32).to(device)
        #query_samples = torch.tensor(query_samples,dtype=torch.float32).to(device)
    elif mode=="Wav2Vec":
        support_samples = torch.tensor(support_samples,dtype=torch.float16).to(device)
        query_samples = torch.tensor(query_samples,dtype=torch.float16).to(device)



    if(mode=="classical_network" or mode=="AST"):
        support_samples=support_samples.view(-1, 1, 128, 2)
        query_samples=query_samples.view(-1, 1, 128, 2)


    if pcen==False or mode=="Wav2Vec":
        mean = support_samples.mean()
        std = support_samples.std()
        support_samples = (support_samples - mean) / std
        query_samples = (query_samples - mean) / std

        min_val = support_samples.min()
        max_val = support_samples.max()
        support_samples = (support_samples - min_val) / (max_val - min_val)
        query_samples = (query_samples - min_val) / (max_val - min_val)
        
        

    if mode=="Wav2Vec":processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

    support_labels = torch.tensor(support_labels).to(device)
    query_labels=torch.tensor(query_labels).to(device)

    """
    print(f'Support Samples:{support_samples.shape}')
    print(f'Support Labels:{support_labels.shape}')

    print(f'Query Samples:{query_samples.shape}')
    print(f'Query labels:{query_labels.shape}')
    """




    support_dataset = TensorDataset(support_samples, support_labels)
    query_dataset = TensorDataset(query_samples,query_labels)

    support_loader = DataLoader(support_dataset, batch_size=batch_size, shuffle=False)
    query_loader = DataLoader(query_dataset, batch_size=batch_size, shuffle=False)

    query_samples_size=query_samples.shape[0]

    del query_samples
    del support_samples
    del support_labels

    aggregated_positive_prototype=None
    aggregated_negative_prototype=None

    with torch.no_grad():
            if rolling_mean==False:
                negative_embeddings=[]
                positive_embeddings=[]
            
            for support_batch,support_labels_batch in support_loader:


                support_batch = support_batch.to(device)
                support_labels_batch=support_labels_batch.to(device)
                
                if rolling_mean==True:
                    negative_embeddings=[]
                    positive_embeddings=[]
               
                if mode=="classical_network":
                    with autocast():
                        for x in range(0,support_batch.shape[0]):

                            frame=support_batch[x].unsqueeze(0)
                            support_embeddings, _, _= model(frame)
                            torch.cuda.empty_cache()
                            if (support_labels_batch[x]==0):
                                negative_embeddings.append(support_embeddings)
                            else:
                                positive_embeddings.append(support_embeddings)
                elif mode=="AST":
                    if support_batch.shape[0]<batch_size:
                        continue
                        
                    support_batch=support_batch.squeeze(dim=1)
                    support_batch = support_batch.permute(0, 2, 1)
                    support_batch = support_batch.reshape(-1, 1024)
                    support_batch = support_batch.unsqueeze(0)
                    with autocast():
                        most_frequent_label = np.bincount(support_labels_batch.cpu()).argmax()
                        batch_embedding,_,_=model(support_batch)

                        if (most_frequent_label==0):
                            negative_embeddings.append(batch_embedding)
                        else:
                            positive_embeddings.append(batch_embedding)
                            #below the code for rolling mean

                elif mode=="Wav2Vec":
                    if support_batch.shape[0]<batch_size:
                        break
                    support_batch = support_batch.view(-1)
                    #the output of processor is a dictionay, using .input_values we are taking the right output.
                    support_batch=processor(support_batch, return_tensors="pt", sampling_rate=16000).input_values
                    support_batch=torch.tensor(support_batch,dtype=torch.half).to(device)
                    with autocast():
                        most_frequent_label = np.bincount(support_labels_batch.cpu()).argmax()
                        torch.cuda.empty_cache()
                        batch_embedding,_,_=model(support_batch)
                    
                        if (most_frequent_label==0):
                            negative_embeddings.append(batch_embedding)
                        else:
                            positive_embeddings.append(batch_embedding)


                if rolling_mean==True:
                    if len(positive_embeddings) > 0:
                        positive_embeddings=torch.stack(positive_embeddings)
                        prototype = positive_embeddings.mean(dim=0)

                        if aggregated_positive_prototype is None:
                            aggregated_positive_prototype=prototype
                        else:
                            aggregated_positive_prototype = (torch.stack([aggregated_positive_prototype, prototype])).mean(dim=0)

                    if len(negative_embeddings) > 0:
                        negative_embeddings=torch.stack(negative_embeddings)
                        prototype = negative_embeddings.mean(dim=0)
                        if aggregated_negative_prototype is None:
                            aggregated_negative_prototype=prototype
                        else:
                            aggregated_negative_prototype = (torch.stack([aggregated_negative_prototype, prototype])).mean(dim=0)
                    torch.cuda.empty_cache()
                    negative_embeddings=[]
                    positive_embeddings=[]
                


            if rolling_mean==False:
                if len(positive_embeddings) > 0:

                    positive_embeddings=torch.stack(positive_embeddings)
                    aggregated_positive_prototype = positive_embeddings.mean(dim=0)

                if len(negative_embeddings) > 0:
                    negative_embeddings=torch.stack(negative_embeddings)
                    aggregated_negative_prototype = negative_embeddings.mean(dim=0)
                
            
                torch.cuda.empty_cache()
                negative_embeddings=[]
                positive_embeddings=[]
            



    #adding the negative and positive prototype for the entire episode.
    negative_prototype=aggregated_negative_prototype
    positive_prototype=aggregated_positive_prototype

    
    progress_bar = tqdm(total=query_samples_size)
    
    with torch.no_grad():
        for query_batch,query_batch_labels in query_loader:
            query_batch=torch.tensor(query_batch)
            query_batch = query_batch.to(device)
            if mode=="classical_network":
                for y in range(0,query_batch.shape[0]):
                    with autocast():
                        frame=query_batch[y]
                        frame=frame.unsqueeze(0)
                        query_embeddings, binary_query_output, multiclass_query_output = model(frame)
                        distance_from_negative = F.pairwise_distance(query_embeddings, negative_prototype)
                        distance_from_positive=F.pairwise_distance(query_embeddings, positive_prototype)
                        if(distance_from_positive<distance_from_negative):
                            query_predictions.append(1)
                        else:
                            query_predictions.append(0)
                        progress_bar.update(1)
            
            elif(mode=="Wav2Vec"):
                if query_batch_labels.shape[0]<batch_size:
                    break
                most_frequent_label = np.bincount(query_batch_labels.cpu()).argmax()
                query_batch = query_batch.view(-1)
                query_batch=processor(query_batch, return_tensors="pt", sampling_rate=16000).input_values
                query_batch=torch.tensor(query_batch,dtype=torch.half).to(device)

                with autocast():
                    query_embeddings,_,multiclass_query_output = model(query_batch)
                distance_from_negative = F.pairwise_distance(query_embeddings,negative_prototype)
                distance_from_positive=F.pairwise_distance(query_embeddings, positive_prototype)
                for h in range (0,15):
                    if(distance_from_positive[0][h]<distance_from_negative[0][h]):
                            query_predictions.append(1)
                            query_predictions.append(1)
                    else:
                            query_predictions.append(0)
                            query_predictions.append(0)
                query_predictions.append(query_predictions[-1])
                query_predictions.append(query_predictions[-1])
            
                progress_bar.update(batch_size)
  
            elif mode=="AST":
                with autocast():
                    if query_batch.shape[0]<batch_size:
                        continue
                    query_batch=query_batch.squeeze(dim=1)
                    query_batch = query_batch.permute(0, 2, 1)
                    query_batch = query_batch.reshape(-1, 1024)
                    query_batch = query_batch.unsqueeze(0)
                    most_frequent_label = np.bincount(query_batch_labels.cpu()).argmax()
                    
                    query_embeddings,_,multiclass_query_output = model(query_batch)
                    most_frequent_label=torch.tensor(most_frequent_label)
                    distance_from_negative = F.pairwise_distance(query_embeddings,negative_prototype)
                    distance_from_positive=F.pairwise_distance(query_embeddings, positive_prototype)
                    if(distance_from_positive<distance_from_negative):
                        for _ in range(0,batch_size):
                            query_predictions.append(1)
                    else:
                        for _ in range(0,batch_size):
                            query_predictions.append(0)

                    progress_bar.update(batch_size)



    query_predictions_np = np.array(query_predictions)
    if len(query_predictions_np)<len(query_labels.cpu().detach().numpy()):
        query_labels=query_labels[:len(query_predictions_np)]

    count_of_ones = np.sum(query_predictions_np)
    count_of_ones_2=np.sum(query_labels.cpu().detach().numpy())
    print(count_of_ones,count_of_ones_2)
    
    

    
    if mode=="AST" or mode=="Wav2Vec":
        sr=16000
    else: 
        sr=22050
    mean_duration=0

    support_set=get_support_set_inference(audio)
    events=support_set[:5]
    for evento in events:
        start=evento['start']
        end=evento['end']
        if mean_duration==0: mean_duration=end-start
        else: mean_duration+=(end-start)
    mean_event_duration=mean_duration/5
    mean_event_duration=mean_duration*0.3

    print(mean_event_duration)


    f1 = f1_score(query_labels.cpu().detach().numpy(), query_predictions_np, average='binary')

    print(f'F1 score pre correzione: {f1}')
    
    query_predictions_np=wrong_prediction_correction(query_predictions_np,mean_event_duration,mode=mode)

    count_of_ones = np.sum(query_predictions_np)
    print(count_of_ones,count_of_ones_2)

    predicted_events=event_finder(query_predictions_np,mean_event_duration,mode=mode)
    real_events=event_finder(query_labels.cpu().detach().numpy(),mean_event_duration,mode=mode)
    print(predicted_events)
    print(real_events)

    tp, fp, fn = compute_tp_fp_fn(predicted_events, real_events)
    metrics = compute_precision_recall_f1(tp, fp, fn)


    f1_segment_based=metrics['f1_score']
    precision_segment_based=metrics['precision']
    recall_segment_based=metrics['recall']
    
    
    f1 = f1_score(query_labels.cpu().detach().numpy(), query_predictions_np, average='binary')
    precision=precision_score(query_labels.cpu().detach().numpy(), query_predictions_np)
    recall=recall_score(query_labels.cpu().detach().numpy(), query_predictions_np)
    

    print(f'F1 score event based: {f1_segment_based}')
    print(f'Precision event based: {precision_segment_based}')
    print(f'Recall event based: {recall_segment_based}')

    print(f'F1 score: {f1}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    return metrics,f1

       


## Inference for an audio given the path to the file and the json file containing its events 

In [126]:
audios=["ME1.wav","file_97_113.wav","file_423_487.wav","R4_cleaned recording_13-10-17.wav","ME2.wav","BUK5_20161101_002104a.wav","BUK5_20180921_015906a.wav","BUK1_20181013_023504.wav","BUK1_20181011_001004.wav",
"BUK4_20161011_000804.wav","BUK4_20171022_004304a.wav","R4_cleaned recording_TEL_20-10-17.wav","R4_cleaned recording_17-10-17.wav","R4_cleaned recording_TEL_24-10-17.wav","R4_cleaned recording_TEL_25-10-17.wav","R4_cleaned recording_16-10-17.wav",
"R4_cleaned recording_TEL_19-10-17.wav","R4_cleaned recording_TEL_23-10-17.wav"]
lista=[1,0]
lista=[0]

for i in lista:
    
    f1_scores_frame_classification_dictionary={}
    f1_scores_dictionary={}
    precision_dictionary={}
    recall_dictionary={}


    #If i=0 AST if i=1 then Wav2Vec. If i=2 CNN with negative support undersampling, if i=3 CNN with full negative support, if i=4 CNN with event spectrogram creation and full negative support.

    number_of_audio_tested=0
    rolling_mean=True

    if i==0:
        audios=["ME1.wav","file_97_113.wav","file_423_487.wav","R4_cleaned recording_13-10-17.wav","BUK5_20161101_002104a.wav","BUK5_20180921_015906a.wav","BUK1_20181013_023504.wav","BUK1_20181011_001004.wav",
        "BUK4_20161011_000804.wav","BUK4_20171022_004304a.wav","R4_cleaned recording_TEL_20-10-17.wav","R4_cleaned recording_17-10-17.wav","R4_cleaned recording_TEL_24-10-17.wav","R4_cleaned recording_TEL_25-10-17.wav","R4_cleaned recording_16-10-17.wav",
        "R4_cleaned recording_TEL_19-10-17.wav","R4_cleaned recording_TEL_23-10-17.wav"]

       
    
    #audios=["R4_cleaned recording_16-10-17.wav"]
    #audios= ["R4_cleaned recording_13-10-17.wav"]
    #note: Pcen can be used only if the encoder is not Wav2Vec.
    pcen=True
    
    if(i==0 or i==1):rolling_mean=False

    if (i==1): pcen=False
    print(i)

    mean_f1_score=0
    mean_precision=0
    mean_recall=0
    mean_f1_score_classification=0

    for audio in audios:
        print(f'Doing inference for audio: {audio}')

        file_name=f'inference_audio{audio}.pth'
    
        

        augment_negative_support=False

        if i==1:
            file_path=os.path.join(directory_path_raw_audio_inference, file_name)
        elif i==0:
            file_path=os.path.join(directory_path_spectrograms_inference_AST, file_name)
        else: 
            if pcen==False:
                file_path=os.path.join(directory_path_spectrograms_inference, file_name)
            else: 
                file_path=os.path.join(directory_path_spectrograms_inference_pcen, file_name)
        print(file_path)


        if os.path.isfile(file_path):
            support_and_query_set=torch.load(file_path)
            print("Audio già pronto per l'inferenza")
        else:
            audio_support_events=get_support_set_inference(audio)
            audio_query_events=get_query_set_inference(audio)
            labels_audio_inference=label_audio(audio_support_events,audio_query_events,audio)
            if i==1:
                support_and_query_set=create_support_and_query_set_inference_audio_raw(labels_audio_inference,audio,audio_support_events)
            elif i==0:
                support_and_query_set=create_support_and_query_set_inference_audio_spectrogramsAST(labels_audio_inference,audio,audio_support_events,pcen=pcen,augment_negative_support=augment_negative_support)
            else:
                support_and_query_set=create_support_and_query_set_inference_audio_spectrograms(labels_audio_inference,audio,audio_support_events,pcen=pcen,augment_negative_support=augment_negative_support)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


        # Dimension for the spectrogram in our cases.
        hidden_dim = 768
        output_dim = 22 



        if i==1:
            mode="Wav2Vec"
            model = CustomWav2Vec2Model(
                base_model_name="facebook/wav2vec2-large-960h-lv60-self",
                num_labels_task_b=22,
            ).to(device)
        elif i==0:
            mode="AST"
            model=AST_Model(output_dim).to(device)
        else:
            input_dim = (128, 2)
            
            mode="classical_network"
            if rolling_mean== True:
                model=PrototypicalNetwork_with_binary_head(input_dim=input_dim,output_dim=output_dim,hidden_dim=hidden_dim).to(device)
            else: 
                model=PrototypicalNetwork(input_dim=input_dim,output_dim=output_dim,hidden_dim=hidden_dim).to(device)

        if i==0:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
            mode="AST"
        elif i==1:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            mode="Wav2Vec"
        else:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            mode="classical_network"   

        if rolling_mean==True:
            rolling="_rolling"
        else: 
            rolling=""
        checkpoint_file=os.path.join(checkpoint_path, "model_type_" + str(mode) +str(rolling)+ "_complete_support_set.pth")
    

        if i==0 or i==1:
            model_info=0
            if os.path.isfile(os.path.join(checkpoint_path, "model_type_" + str(mode)+".pth")):
                checkpoint_file=os.path.join(checkpoint_path, "model_type_" + str(mode) + ".pth")
                print(checkpoint_file)
                model_info=torch.load(checkpoint_file)
        elif i==2:
            model_info=0
            if os.path.isfile(os.path.join(checkpoint_path, "model_type_" + str(mode) +str(rolling)+"_balanced_support_set.pth")):
                checkpoint_file=os.path.join(checkpoint_path, "model_type_" + str(mode) +str(rolling)+ "_balanced_support_set.pth")
                model_info=torch.load(checkpoint_file)
        elif i==3:
            model_info=0
            if os.path.isfile(os.path.join(checkpoint_path, "model_type_" + str(mode) +str(rolling)+"_complete_support_set.pth")):
                checkpoint_file=os.path.join(checkpoint_path, "model_type_" + str(mode) +str(rolling)+ "_complete_support_set.pth")
                model_info=torch.load(checkpoint_file)
        elif i==4:
            model_info=0
            if os.path.isfile(os.path.join(checkpoint_path, "model_type_" + str(mode) +str(rolling)+ "_event_spectrogram.pth")):
                checkpoint_file=os.path.join(checkpoint_path, "model_type_" + str(mode) +str(rolling)+ "_event_spectrogram.pth")
                model_info=torch.load(checkpoint_file)
        else: 
            model_info=0


        metrics,f1_score_classification=inference_audio(model=model,audio_name=audio,model_checkpoint=model_info,mode=mode,pcen=pcen)

        
        f1_scores_frame_classification_dictionary[audio]=f1_score_classification
        f1_scores_dictionary[audio]=metrics['f1_score']
        precision_dictionary[audio]=metrics['precision']
        recall_dictionary[audio]=metrics['recall']


        
        if(mean_f1_score==0): mean_f1_score=metrics['f1_score']
        else: mean_f1_score+=metrics['f1_score']


        if(mean_precision==0): mean_precision=metrics['precision']
        else: mean_precision+=metrics['precision']

        if(mean_recall==0): mean_recall=metrics['recall']
        else: mean_recall+=metrics['recall']

        if(mean_f1_score_classification==0):mean_f1_score_classification=f1_score_classification
        else: mean_f1_score_classification+=f1_score_classification
        number_of_audio_tested+=1

    mean_f1_score=mean_f1_score/number_of_audio_tested
    mean_recall=mean_recall/number_of_audio_tested
    mean_precision=mean_precision/number_of_audio_tested
    mean_f1_score_classification=mean_f1_score_classification/number_of_audio_tested
    


    print(f'Mean F1 score for frame classification:{mean_f1_score_classification}')
    print(f'Mean F1 score:{mean_f1_score}')
    print(f'Mean recall:{mean_recall}')
    print(f'Mean precision:{mean_precision}')

    if not os.path.exists(directory_path_inference_stats):
            os.makedirs(directory_path_inference_stats)
    torch.save({'f1_scores_frame_classification':f1_scores_frame_classification_dictionary,
                'mean_f1_score_classification':mean_f1_score_classification,
                'f1_scores':f1_scores_dictionary,
                'mean_f1_score':mean_f1_score,
                'precisions':precision_dictionary,
                'mean_precision':mean_precision,
                'recalls':recall_dictionary,
                'mean_recall':mean_recall
    },os.path.join(directory_path_inference_stats, "inference_stats_" + str(mode) +str(i)+ ".pth"))



0
Doing inference for audio: ME1.wav
E:/Tesi/spectrograms_support_and_query_for_inference_AST_16KHz_pcen_pth\inference_audioME1.wav.pth
Audio già pronto per l'inferenza
E:/Tesi/Models_pth\model_type_AST.pth
0
torch.Size([4350, 128, 2]) torch.Size([4350])


C:\Users\aldob\AppData\Local\Temp\ipykernel_11328\1557730710.py:96: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  support_labels = torch.tensor(support_labels).to(device)
  0%|          | 0/47196 [00:00<?, ?it/s]C:\Users\aldob\AppData\Local\Temp\ipykernel_11328\1557730710.py:237: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  query_batch=torch.tensor(query_batch)
 98%|█████████▊| 46080/47196 [00:03<00:00, 13508.01it/s]C:\Users\aldob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted sampl

0 322
0.3650999999999996
F1 score pre correzione: 0.0
0 322
36
min_event_frame:10
36
min_event_frame:10
[]
[(9.46, 9.950000000000001), (12.32, 12.870000000000001), (31.18, 31.77), (32.58, 33.15), (35.88, 36.27), (38.160000000000004, 38.65)]
0 0 6
F1 score event based: 0.0
Precision event based: 1e-05
Recall event based: 0.0
F1 score: 0.0
Precision: 0.0
Recall: 0.0
Doing inference for audio: file_97_113.wav
E:/Tesi/spectrograms_support_and_query_for_inference_AST_16KHz_pcen_pth\inference_audiofile_97_113.wav.pth
Audio già pronto per l'inferenza
E:/Tesi/Models_pth\model_type_AST.pth
0
torch.Size([117846, 128, 2]) torch.Size([117846])


C:\Users\aldob\AppData\Local\Temp\ipykernel_11328\1557730710.py:96: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  support_labels = torch.tensor(support_labels).to(device)
  0%|          | 0/67275 [00:00<?, ?it/s]C:\Users\aldob\AppData\Local\Temp\ipykernel_11328\1557730710.py:237: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  query_batch=torch.tensor(query_batch)
100%|█████████▉| 67072/67275 [00:05<00:00, 12042.69it/s]


2048 27158
12.058792199999997
F1 score pre correzione: 0.010203382866534274
2048 27158
1205
min_event_frame:10
1205
min_event_frame:10
[(30.72, 35.83), (307.2, 312.31), (568.32, 573.4300000000001), (614.4, 619.51)]
[(6.0600000000000005, 31.27), (48.44, 68.35000000000001), (74.0, 112.77), (119.56, 134.75), (140.1, 147.15), (152.88, 161.79), (167.14000000000001, 183.87), (190.14000000000001, 197.51), (203.42000000000002, 259.77), (268.02, 306.59000000000003), (313.0, 316.31), (342.14, 343.77), (361.34000000000003, 363.27), (424.44, 424.81), (448.44, 450.79), (457.92, 463.11), (493.92, 494.57), (508.48, 509.01), (514.4, 517.55), (523.38, 524.51), (531.24, 533.17), (536.92, 538.83), (547.22, 549.45), (572.04, 572.95), (625.1800000000001, 627.29), (642.3000000000001, 644.71), (657.4, 658.23), (663.5, 667.87)]
2 2 26
F1 score event based: 0.125
Precision event based: 0.5
Recall event based: 0.07142857142857142
F1 score: 0.010203382866534274
Precision: 0.07275390625
Recall: 0.0054864128433610

...

43008 31073
19.9852653
F1 score pre correzione: 0.7973704458633117
43008 31073
1998
min_event_frame:10
1998
min_event_frame:10
[(5.12, 158.71), (163.84, 322.55), (327.68, 409.59000000000003), (414.72, 450.55)]
[(49.32, 59.47), (69.76, 103.29), (115.38, 183.35), (197.70000000000002, 226.57), (240.1, 291.07), (308.14, 367.03000000000003), (390.36, 450.55)]
4 0 3
F1 score event based: 0.7272727272727273
Precision event based: 1.0
Recall event based: 0.5714285714285714
F1 score: 0.7973704458633117
Precision: 0.6867327008928571
Recall: 0.9505036526888295
Doing inference for audio: R4_cleaned recording_TEL_19-10-17.wav
E:/Tesi/spectrograms_support_and_query_for_inference_AST_16KHz_pcen_pth\inference_audioR4_cleaned recording_TEL_19-10-17.wav.pth
Audio già pronto per l'inferenza
E:/Tesi/Models_pth\model_type_AST.pth
0
torch.Size([197212, 128, 2]) torch.Size([197212])


C:\Users\aldob\AppData\Local\Temp\ipykernel_11328\1557730710.py:96: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  support_labels = torch.tensor(support_labels).to(device)
  0%|          | 0/57000 [00:00<?, ?it/s]C:\Users\aldob\AppData\Local\Temp\ipykernel_11328\1557730710.py:237: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  query_batch=torch.tensor(query_batch)
100%|█████████▉| 56832/57000 [00:04<00:00, 12986.24it/s]


26112 46656
19.6003188
F1 score pre correzione: 0.5663478452066842
26112 46656
1960
min_event_frame:10
1960
min_event_frame:10
[(15.36, 35.83), (40.96, 46.07), (51.2, 56.31), (66.56, 71.67), (81.92, 97.27), (102.4, 112.63), (117.76, 127.99000000000001), (133.12, 138.23), (143.36, 148.47), (158.72, 163.83), (168.96, 174.07), (184.32, 189.43), (194.56, 199.67000000000002), (215.04, 225.27), (235.52, 240.63), (245.76, 250.87), (256.0, 261.11), (266.24, 271.35), (276.48, 281.59000000000003), (291.84000000000003, 302.07), (307.2, 312.31), (317.44, 327.67), (337.92, 343.03000000000003), (358.40000000000003, 363.51), (373.76, 378.87), (389.12, 399.35), (414.72, 419.83), (424.96000000000004, 435.19), (440.32, 445.43), (450.56, 455.67), (486.40000000000003, 491.51), (496.64, 506.87), (512.0, 517.11), (522.24, 537.59), (547.84, 552.95), (563.2, 568.3100000000001)]
[(4.3, 25.67), (31.62, 127.63000000000001), (137.44, 167.47), (173.6, 203.37), (213.5, 216.73000000000002), (227.54, 277.71), (282.2,

C:\Users\aldob\AppData\Local\Temp\ipykernel_11328\1557730710.py:96: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  support_labels = torch.tensor(support_labels).to(device)
  0%|          | 0/70560 [00:00<?, ?it/s]C:\Users\aldob\AppData\Local\Temp\ipykernel_11328\1557730710.py:237: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  query_batch=torch.tensor(query_batch)
 99%|█████████▉| 70144/70560 [00:05<00:00, 13022.56it/s]

68608 57683
13.860858300000004
F1 score pre correzione: 0.8898971423141792
68608 57683
1386
min_event_frame:10
1386
min_event_frame:10
[(0.0, 61.43), (66.56, 235.51), (240.64000000000001, 686.07), (691.2, 701.4300000000001)]
[(4.46, 21.73), (25.0, 52.31), (57.0, 68.99), (73.12, 105.31), (108.66, 124.91), (128.26, 157.01), (160.26, 192.05), (197.56, 212.45000000000002), (215.72, 227.03), (231.68, 253.35), (263.18, 276.33), (280.08, 294.81), (298.24, 333.55), (337.40000000000003, 353.95), (358.76, 389.89), (393.44, 404.15000000000003), (407.78000000000003, 421.33), (428.06, 455.75), (459.58, 478.73), (482.16, 511.65000000000003), (516.5, 535.73), (539.58, 551.53), (555.66, 576.99), (580.98, 600.79), (604.4, 617.91), (622.14, 632.73), (636.5, 665.71), (671.22, 690.73), (695.22, 701.4300000000001)]
4 0 25
F1 score event based: 0.2424242424242424
Precision event based: 1.0
Recall event based: 0.13793103448275862
F1 score: 0.8898971423141792
Precision: 0.8190444263059702
Recall: 0.9741691659

In [121]:
stats=torch.load(os.path.join(directory_path_inference_stats, "inference_stats_Wav2Vec1.pth"))
print(stats)

{'f1_scores_frame_classification': {'ME1.wav': 0.010654617465762977, 'file_97_113.wav': 0.4763845156510465, 'file_423_487.wav': 0.8496466737971825, 'R4_cleaned recording_13-10-17.wav': 0.4195501083626037, 'ME2.wav': 0.07405558329172908, 'BUK5_20161101_002104a.wav': 0.024970553592461718, 'BUK5_20180921_015906a.wav': 0.0030257186081694403, 'BUK1_20181013_023504.wav': 0.001815757369421511, 'BUK1_20181011_001004.wav': 0.011139900939488481, 'BUK4_20161011_000804.wav': 0.0012488477892420684, 'BUK4_20171022_004304a.wav': 0.007628196072891651, 'R4_cleaned recording_TEL_20-10-17.wav': 0.4021360712810527, 'R4_cleaned recording_17-10-17.wav': 0.9330605581996841, 'R4_cleaned recording_TEL_24-10-17.wav': 0.48888824584756063, 'R4_cleaned recording_TEL_25-10-17.wav': 0.5311763874493546, 'R4_cleaned recording_16-10-17.wav': 0.8152404655588903, 'R4_cleaned recording_TEL_19-10-17.wav': 0.7905480265548691, 'R4_cleaned recording_TEL_23-10-17.wav': 0.5355309291259704}, 'mean_f1_score_classification': 0.354

In [119]:
stats=torch.load(os.path.join(directory_path_inference_stats, "inference_stats_classical_network3.pth"))
#print(stats['f1_scores'])
print (stats)

{'f1_scores_frame_classification': {'ME1.wav': 0.009254431699687175, 'file_97_113.wav': 0.4763275382110405, 'file_423_487.wav': 0.6456009307509557, 'R4_cleaned recording_13-10-17.wav': 0.6684264278419357, 'ME2.wav': 0.04065654183567588, 'BUK5_20161101_002104a.wav': 0.01728683246137397, 'BUK5_20180921_015906a.wav': 0.0025381464109719173, 'BUK1_20181013_023504.wav': 0.0014033075808788024, 'BUK1_20181011_001004.wav': 0.011122844068989793, 'BUK4_20161011_000804.wav': 0.0014972008853013931, 'BUK4_20171022_004304a.wav': 0.0013636594921894877, 'R4_cleaned recording_TEL_20-10-17.wav': 0.6803912782488861, 'R4_cleaned recording_17-10-17.wav': 0.6892615575173955, 'R4_cleaned recording_TEL_24-10-17.wav': 0.7008663486810085, 'R4_cleaned recording_TEL_25-10-17.wav': 0.7048649578900947, 'R4_cleaned recording_16-10-17.wav': 0.64754523239454, 'R4_cleaned recording_TEL_19-10-17.wav': 0.6733742870483114, 'R4_cleaned recording_TEL_23-10-17.wav': 0.7109011462944047}, 'mean_f1_score_classification': 0.37126

In [120]:
stats=torch.load(os.path.join(directory_path_inference_stats, "inference_stats_classical_network4.pth"))
#print(stats['f1_scores'])
print (stats)

{'f1_scores_frame_classification': {'ME1.wav': 0.010837266212200665, 'file_97_113.wav': 0.47370578349797104, 'file_423_487.wav': 0.656851399842463, 'R4_cleaned recording_13-10-17.wav': 0.6847767200681062, 'ME2.wav': 0.03879939738092479, 'BUK5_20161101_002104a.wav': 0.016633299715502838, 'BUK5_20180921_015906a.wav': 0.0024706518719939184, 'BUK1_20181013_023504.wav': 0.0014030852789197752, 'BUK1_20181011_001004.wav': 0.011075902508366157, 'BUK4_20161011_000804.wav': 0.0016787708477792782, 'BUK4_20171022_004304a.wav': 0.0012442199483741574, 'R4_cleaned recording_TEL_20-10-17.wav': 0.6968995528121397, 'R4_cleaned recording_17-10-17.wav': 0.7122943463954532, 'R4_cleaned recording_TEL_24-10-17.wav': 0.7064389910592846, 'R4_cleaned recording_TEL_25-10-17.wav': 0.7131405302263109, 'R4_cleaned recording_16-10-17.wav': 0.6572886297376094, 'R4_cleaned recording_TEL_19-10-17.wav': 0.6601986687527194, 'R4_cleaned recording_TEL_23-10-17.wav': 0.7419991716139721}, 'mean_f1_score_classification': 0.37

In [117]:
stats=torch.load(os.path.join(directory_path_inference_stats, "inference_stats_AST0.pth"))
#print(stats['f1_scores'])
print (stats)

{'f1_scores_frame_classification': {'file_423_487.wav': 0.002643042435514659}, 'mean_f1_score_classification': 0.002643042435514659, 'f1_scores': {'file_423_487.wav': 0.0}, 'mean_f1_score': 0.0, 'precisions': {'file_423_487.wav': 1e-05}, 'mean_precision': 1e-05, 'recalls': {'file_423_487.wav': 0.0}, 'mean_recall': 0.0}


 'recalls': {'ME1.wav': 0.16666666666666666, 'file_97_113.wav': 0.5365853658536586, 'file_423_487.wav': 0.4827586206896552, 'R4_cleaned recording_13-10-17.wav': 0.1, 'ME2.wav': 0.10714285714285714, 'BUK5_20161101_002104a.wav': 0.0, 'BUK5_20180921_015906a.wav': 0.16666666666666666, 'BUK1_20181013_023504.wav': 1.0, 'BUK1_20181011_001004.wav': 0.0, 'BUK4_20161011_000804.wav': 0.0, 'BUK4_20171022_004304a.wav': 0.0, 'R4_cleaned recording_TEL_20-10-17.wav': 0.42424242424242425, 'R4_cleaned recording_17-10-17.wav': 0.45, 'R4_cleaned recording_TEL_24-10-17.wav': 0.6363636363636364, 'R4_cleaned recording_TEL_25-10-17.wav': 0.5957446808510638, 'R4_cleaned recording_16-10-17.wav': 0.3333333333333333, 'R4_cleaned recording_TEL_19-10-17.wav': 0.44, 'R4_cleaned recording_TEL_23-10-17.wav': 0.7692307692307693}, 'mean_recall': 0.34492972339115174}